# Hybrid
Going to attempt a hybrid model after the example of [this Teck Meng Wong notebook](https://www.kaggle.com/teckmengwong/tps2201-hybrid-time-series/notebook).

- 20220122: Going to try to form ensembles, with more code architecture. Forecasting models will include Prophet, NeuralProphet, Ridge, and Linear (with more to come -- e.g. perhaps transformers and other DNNs); residual models will include GBMs, perhaps some tabular DNNs too.

In [1]:
# notebook configuration
# if '/sf/' in pwd:
#     COLAB, SAGE = False, False
# elif 'google.colab' in str(get_ipython()):
#     COLAB, SAGE = True, False # do colab-specific installs later
# else:
#     COLAB, SAGE = False, True
    
CONTEXT = 'local' # or 'colab', 'sage', 'kaggle'
USE_GPU = True 
%config Completer.use_jedi = False

## Imports

In [2]:
# basic imports
from pathlib import Path
import os
import math
from datetime import datetime
import random

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

import requests # for telegram notifications
from tqdm.notebook import tqdm

from joblib import dump, load

import datetime as dt

Now, non-stdlib imports

In [3]:
# model selection
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold

# metrics
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error

# normalization
# from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler, QuantileTransformer
# from gauss_rank_scaler import GaussRankScaler

# feature generation
# import category_encoders as ce

# models
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
# from sklearn.ensemble import StackingClassifier, RandomForestClassifier
import torch
from torch.optim import Adam, AdamW, Adagrad, SGD, RMSprop, LBFGS
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingWarmRestarts, CyclicLR, OneCycleLR, StepLR, CosineAnnealingLR
# from pytorch_widedeep import Trainer
# from pytorch_widedeep.preprocessing import WidePreprocessor, TabPreprocessor
# from pytorch_widedeep.models import Wide, TabMlp, WideDeep, SAINT#, TabTransformer, TabNet, TabFastFormer, TabResnet
# from pytorch_widedeep.metrics import Accuracy
# from pytorch_widedeep.callbacks import EarlyStopping, LRHistory, ModelCheckpoint

# feature reduction
# from sklearn.decomposition import PCA
# from umap import UMAP

# clustering
# from sklearn.cluster import DBSCAN, KMeans
# import hdbscan

# feature selection
# from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
# import featuretools as ft
# from BorutaShap import BorutaShap
# from boruta import BorutaPy

# tracking 
import wandb
from wandb.xgboost import wandb_callback
from wandb.lightgbm import wandb_callback
os.environ['WANDB_NOTEBOOK_NAME'] = f"nb_{datetime.now().strftime('%Y%m%d')}.ipynb"

In [4]:
# # time series
# import tsfresh

# import darts
# from darts import TimeSeries
# from darts.models import ExponentialSmoothing, AutoARIMA, ARIMA, Prophet, RandomForest, RegressionEnsembleModel, RegressionModel, TFTModel, TCNModel, TransformerModel, NBEATSModel
import holidays
import dateutil.easter as easter
from prophet import Prophet
from neuralprophet import NeuralProphet

## Routing

Now, datapath setup

In [5]:
if CONTEXT == 'colab':
    # mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')
    
    # handling datapath
    # datapath = Path('/content/drive/MyDrive/kaggle/tabular_playgrounds/dec2021/')
    root = Path('') # TODO

elif CONTEXT == 'sage':
    root = Path('') # TODO
    
elif CONTEXT == 'kaggle':
    root = Path('') # TODO
    
else: # if on local machine
    root = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/jan2022/')
    datapath = root/'datasets'
    # edapath = root/'EDA'
    # modelpath = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/oct2021/models/')
    predpath = root/'preds'
    subpath = root/'submissions'
    studypath = root/'studies'
    
    for pth in [datapath, predpath, subpath, studypath]:
        pth.mkdir(exist_ok=True)

## Helpers

In [6]:
SEED = 42

# Function to seed everything but the models
def seed_everything(seed, pytorch=True, reproducible=True):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    if pytorch:
        torch.manual_seed(seed) # set torch CPU seed
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(seed) # set torch GPU(s) seed(s)
        if reproducible and torch.backends.cudnn.is_available():
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False

seed_everything(seed=SEED)

In [7]:
def reduce_memory_usage(df, verbose=True):
    """
    Function to reduce memory usage by downcasting datatypes in a Pandas DataFrame when possible.
    
    h/t to Bryan Arnold (https://www.kaggle.com/puremath86/label-correction-experiments-tps-nov-21)
    """
    
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [8]:
tg_api_token = 'your_api_token' # for Galileo (jupyter_watcher_bot) on Telegram
tg_chat_id = 'your_chat_id'

import requests

def send_tg_message(text='Cell execution completed.'):  
    """
    h/t Ivan Dembicki Jr. for the base version 
    (https://medium.com/@ivan.dembicki.jr/notifications-in-jupyter-notebook-with-telegram-f2e892c55173)
    """
    requests.post('https://api.telegram.org/' +  'bot{}/sendMessage'.format(tg_api_token),
                  params=dict(chat_id=tg_chat_id, text=text))

In [9]:
def SMAPE(y_true, y_pred):
    '''
    h/t Jean-François Puget (@CPMP) -- see https://www.kaggle.com/c/web-traffic-time-series-forecasting/discussion/36414
    '''
    denominator = (y_true + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return np.mean(diff)

In [10]:
# https://www.kaggle.com/c/ventilator-pressure-prediction/discussion/282735
def better_than_median(inputs, axis):
    """Compute the mean of the predictions if there are no outliers,
    or the median if there are outliers.

    Parameter: inputs = ndarray of shape (n_samples, n_folds)"""
    spread = inputs.max(axis=axis) - inputs.min(axis=axis) 
    spread_lim = 0.45
    print(f"Inliers:  {(spread < spread_lim).sum():7} -> compute mean")
    print(f"Outliers: {(spread >= spread_lim).sum():7} -> compute median")
    print(f"Total:    {len(inputs):7}")
    return np.where(spread < spread_lim,
                    np.mean(inputs, axis=axis),
                    np.median(inputs, axis=axis))

In [11]:
# https://www.kaggle.com/teckmengwong/tps2201-hybrid-time-series
def plot_periodogram(ts, detrend='linear', ax=None):
    from scipy.signal import periodogram
    fs = pd.Timedelta("1Y") / pd.Timedelta("1D")
    freqencies, spectrum = periodogram(
        ts,
        fs=fs,
        detrend=detrend,
        window="boxcar",
        scaling='spectrum',
    )
    if ax is None:
        _, ax = plt.subplots()
    ax.step(freqencies, spectrum, color="purple")
    ax.set_xscale("log")
    ax.set_xticks([1, 2, 4, 6, 12, 26, 52, 104])
    ax.set_xticklabels(
        [
            "Annual (1)",
            "Semiannual (2)",
            "Quarterly (4)",
            "Bimonthly (6)",
            "Monthly (12)",
            "Biweekly (26)",
            "Weekly (52)",
            "Semiweekly (104)",
        ],
        rotation=30,
    )
    ax.ticklabel_format(axis="y", style="sci", scilimits=(0, 0))
    ax.set_ylabel("Variance")
    ax.set_title("Periodogram")
    return ax

In [12]:
# https://www.kaggle.com/teckmengwong/tps2201-hybrid-time-series
def fourier_features(index, freq, order):
    time = np.arange(len(index), dtype=np.float32)
    k = 2 * np.pi * (1 / freq) * time
    features = {}
    for i in range(1, order + 1):
        features.update({
            f"sin_{freq}_{i}": np.sin(i * k),
            f"cos_{freq}_{i}": np.cos(i * k),
        })
    return pd.DataFrame(features, index=index)

## Dataset Setup

### Original Data Loading

In [13]:
# dataset_params will initially include either trivial class instances or loaded, precomputed artifacts
dataset_params = {
    'train_source': str(datapath/'train.csv'),
    'target_source': str(datapath/'train.csv'),
    'test_source': str(datapath/'test.csv'),
    # 'scaler': str(RobustScaler()),
    # 'pca': str(load(datapath/'pca_mle-RobustScaled_orig_trainset.joblib')),
    # 'umap': str(load(datapath/'umap_reducer-20211107-n_comp10-n_neighbors15-rs42-pca_mle-RobustScaled_orig_trainset.joblib')),
}   

# referring back to the already-entered attributes, specify how the pipeline was sequenced
# dataset_params['preprocessing_pipeline'] = str([dataset_params['scaler'], dataset_params['pca'], dataset_params['umap']]) # ACTUALLY this is unwieldy
# dataset_params['preprocessing_pipeline'] = '[scaler, pca, umap]' # more fragile, but also more readable

# now, load the datasets and generate more metadata from them
train_df = pd.read_csv(datapath/'train.csv')
test_df = pd.read_csv(datapath/'test.csv')
orig_train_df = train_df.copy()
orig_test_df = test_df.copy()

Since the dates are natively `Object` dtype (i.e. strings), we have to convert them:

In [14]:
# https://www.kaggle.com/ambrosm/tpsjan22-03-linear-model
for df in [train_df, test_df]:
    df['date'] = pd.to_datetime(df.date)

# for convenience later
countries = ['Sweden', 'Finland', 'Norway']
stores = ['KaggleMart', 'KaggleRama']
products = ['Kaggle Mug', 'Kaggle Hat', 'Kaggle Sticker']

Provisionally, I'm going to concatenate together the `train_df` and `test_df` for preprocessing, to avoid having to constantly apply transforms twice (since I don't anticipate doing any transforms that might allow data leakage to occur).

In [15]:
all_df = pd.concat([train_df, test_df], axis=0)
# all_df.columns
print(len(all_df) == len(train_df) + len(test_df))
del train_df, test_df

True


### GDP Data
Here's data from Carl McBride Ellis ([notebook](https://www.kaggle.com/carlmcbrideellis/gdp-of-finland-norway-and-sweden-2015-2019) and [dataset](https://www.kaggle.com/carlmcbrideellis/gdp-20152019-finland-norway-and-sweden) for doing GDP comparisons. They're frequently used in other entries. I've created a function to add them on.

In [16]:
def add_gdp_data(df):
    gdp_df = pd.read_csv(datapath/'GDP_data_2015_to_2019_Finland_Norway_Sweden.csv')
    gdp_df.set_index('year', inplace=True)
    def get_gdp(row):
        country = 'GDP_' + row.country
        return gdp_df.loc[row.date.year, country]

    df['gdp'] = np.log1p(df.apply(get_gdp, axis=1))
    return df

I'll also define here (but perhaps move later) the GDP exponent, which will be used to transform the targets before inference (dividing num_sold by the $GDP^{1.212}$ and then taking the logarithm (after @ambrosm)

In [17]:
gdp_exponent = 1.2121103201489674 # see https://www.kaggle.com/ambrosm/tpsjan22-03-linear-model for an explanation

In [18]:
all_df = add_gdp_data(all_df)

In [19]:
all_df

,row_id,date,country,store,product,num_sold,gdp
0,0,2015-01-01,Finland,KaggleMart,Kaggle Mug,329.0,5.461456
1,1,2015-01-01,Finland,KaggleMart,Kaggle Hat,520.0,5.461456
2,2,2015-01-01,Finland,KaggleMart,Kaggle Sticker,146.0,5.461456
3,3,2015-01-01,Finland,KaggleRama,Kaggle Mug,572.0,5.461456
4,4,2015-01-01,Finland,KaggleRama,Kaggle Hat,911.0,5.461456
...,...,...,...,...,...,...,...
6565,32863,2019-12-31,Sweden,KaggleMart,Kaggle Hat,NaN,6.282042
6566,32864,2019-12-31,Sweden,KaggleMart,Kaggle Sticker,NaN,6.282042
6567,32865,2019-12-31,Sweden,KaggleRama,Kaggle Mug,NaN,6.282042
6568,32866,2019-12-31,Sweden,KaggleRama,Kaggle Hat,NaN,6.282042


## Feature Engineering

### Time Features

The goal of this function is to create features that will capture seasonalities -- but **not** trends. The trends will (hopefully) be captured by the deployment of linear forecasting algorithms on raw time series data (consisting exclusively of dates and targets); we want to have seasonalities that the residual models can learn, however -- holidays, weekly patterns, climactic season patterns, etc.

The cell below will generate the `holidays` library's entries for the three countries. I may want to follow the template of @teckmengwong's code below, and add more holidays -- then, do some feature importance checking, and perhaps whittle down the features accordingly.

In [20]:
for c in [holidays.Finland, holidays.Sweden, holidays.Norway]:
#     print(c)
    for h in c(years = [2019], observed=True).items():
#         print(h)
        pass

In [21]:
def temporal_engineering(df):
    '''
    Function inspired by / borrowing from @teckmengwong and @ambrosm to create time features that will
    capture seasonality.
    '''
    
#     df[YEAR] = df[DATE].dt.year
    df['month'] = df['date'].dt.month
#     df['week'] = df['date'].dt.week # not used by Teck Meng Wong
#     df['day'] = df['date'].dt.day # not used by Teck Meng Wong
#     df['day_of_year'] = df['date'].dt.dayofyear # not used by Teck Meng Wong
#     df['day_of_month'] = df['date'].dt.days_in_month # not used by Teck Meng Wong
#     df['day_of_week'] = df['date'].dt.dayofweek # not used by Teck Meng Wong
#    df['weekday'] = df['date'].dt.weekday # not used by Teck Meng Wong
    # Teck Meng Wong mapped the integers to first-letters in triplets
    # I'm leaving it as integers, where winter=1, spring=2, summer=3, fall=4
    df['season'] = ((df['date'].dt.month % 12 + 3) // 3) #.map({1:'DJF', 2: 'MAM', 3:'JJA', 4:'SON'})
#     df['month'] = df['month'].apply(lambda x: calendar.month_abbr[x])

    df['wd4'] = df['date'].dt.weekday == 4
    df['wd56'] = df['date'].dt.weekday >= 5
#     df['wd6'] = df['date'].dt.weekday >= 6
#     df.loc[(df.date.dt.year != 2016) & (df.date.dt.month >=3), 'day_of_year'] += 1 # fix for leap years
    
    # 21 days cyclic for lunar
    dayofyear = df.date.dt.dayofyear # for convenience
    
    # here he's creating Fourier features
    for k in range(1, 32, 4):
        df[f'sin{k}'] = np.sin(dayofyear / 365 * 2 * math.pi * k)
        df[f'cos{k}'] = np.cos(dayofyear / 365 * 2 * math.pi * k)
        df[f'finland_sin{k}'] = np.where(df['country'] == 'Finland', df[f'sin{k}'], 0)
        df[f'finland_cos{k}'] = np.where(df['country'] == 'Finland', df[f'cos{k}'], 0)
        df[f'norway_sin{k}'] = np.where(df['country'] == 'Norway', df[f'sin{k}'], 0)
        df[f'norway_cos{k}'] = np.where(df['country'] == 'Norway', df[f'cos{k}'], 0)
        df[f'store_sin{k}'] = np.where(df['store'] == 'KaggleMart', df[f'sin{k}'], 0)
        df[f'store_cos{k}'] = np.where(df['store'] == 'KaggleMart', df[f'cos{k}'], 0)
        df[f'mug_sin{k}'] = np.where(df['product'] == 'Kaggle Mug', df[f'sin{k}'], 0)
        df[f'mug_cos{k}'] = np.where(df['product'] == 'Kaggle Mug', df[f'cos{k}'], 0)
        df[f'sticker_sin{k}'] = np.where(df['product'] == 'Kaggle Sticker', df[f'sin{k}'], 0)
        df[f'sticker_cos{k}'] = np.where(df['product'] == 'Kaggle Sticker', df[f'cos{k}'], 0)
    
#     df[f'semiweekly_sin'] = np.sin(dayofyear / 365 * 2 * math.pi * 14)
#     df[f'semiweekly_cos'] = np.cos(dayofyear / 365 * 2 * math.pi * 14)
#     df[f'lunar_sin'] = np.sin(dayofyear / 365 * 2 * math.pi * 21)
#     df[f'lunar_cos'] = np.cos(dayofyear / 365 * 2 * math.pi * 21)
    df[f'season_sin'] = np.sin(dayofyear / 365 * 2 * math.pi * 91.5)
    df[f'season_cos'] = np.cos(dayofyear / 365 * 2 * math.pi * 91.5)
#     df = pd.concat([df, pd.DataFrame({f'fin{ptr[1]}':
#                                       (df.date == pd.Timestamp(ptr[0])) & (df.country == 'Finland')
#                                       for ptr in holidays.Finland(years = [2015,2016,2017,2018,2019]).items()})], axis=1)
#     df = pd.concat([df, pd.DataFrame({f'nor{ptr[1]}':
#                                       (df.date == pd.Timestamp(ptr[0])) & (df.country == 'Norway')
#                                       for ptr in holidays.Norway(years = [2015,2016,2017,2018,2019]).items()})], axis=1)
#     df = pd.concat([df, pd.DataFrame({f'swe{ptr[1]}':
#                                       (df.date == pd.Timestamp(ptr[0])) & (df.country == 'Sweden')
#                                       for ptr in holidays.Sweden(years = [2015,2016,2017,2018,2019]).items()})], axis=1)

    # End of year
    # Dec - teckmengwong
    for d in range(24, 32):
        df[f"dec{d}"] = (df.date.dt.month == 12) & (df.date.dt.day == d)
    # I'm unsure of the logic of only doing this for Norway
    for d in range(24, 32):
        df[f"n-dec{d}"] = (df.date.dt.month == 12) & (df.date.dt.day == d) & (df.country == 'Norway')
    
    # not sure why he's using different date ranges for each country here
    # Jan - teckmengwong
    for d in range(1, 14):
        df[f"f-jan{d}"] = (df.date.dt.month == 1) & (df.date.dt.day == d) & (df.country == 'Finland')
    for d in range(1, 10):
        df[f"n-jan{d}"] = (df.date.dt.month == 1) & (df.date.dt.day == d) & (df.country == 'Norway')
    for d in range(1, 15):
        df[f"s-jan{d}"] = (df.date.dt.month == 1) & (df.date.dt.day == d) & (df.country == 'Sweden')
    
    
    # May - tekcmengwong
    for d in list(range(1, 10)): # May Day and after, I guess
        df[f"may{d}"] = (df.date.dt.month == 5) & (df.date.dt.day == d)
    for d in list(range(19, 26)):
        df[f"may{d}"] = (df.date.dt.month == 5) & (df.date.dt.day == d) & (df.country == 'Norway')
    # June 
    for d in list(range(8, 14)):
        df[f"june{d}"] = (df.date.dt.month == 6) & (df.date.dt.day == d) & (df.country == 'Sweden')
    
    #Swedish Rock Concert - teckmengwong
    #Jun 3, 2015 – Jun 6, 2015
    #Jun 8, 2016 – Jun 11, 2016
    #Jun 7, 2017 – Jun 10, 2017
    #Jun 6, 2018 – Jun 10, 2018
    #Jun 5, 2019 – Jun 8, 2019
    swed_rock_fest  = df.date.dt.year.map({2015: pd.Timestamp(('2015-06-6')),
                                         2016: pd.Timestamp(('2016-06-11')),
                                         2017: pd.Timestamp(('2017-06-10')),
                                         2018: pd.Timestamp(('2018-06-10')),
                                         2019: pd.Timestamp(('2019-06-8'))})

    df = pd.concat([df, pd.DataFrame({f"swed_rock_fest{d}":
                                      (df.date - swed_rock_fest == np.timedelta64(d, "D")) & (df.country == 'Sweden')
                                      for d in list(range(-3, 3))})], axis=1)

    
    # Last Wednesday of June - teckmengwong
    wed_june_date = df.date.dt.year.map({2015: pd.Timestamp(('2015-06-24')),
                                         2016: pd.Timestamp(('2016-06-29')),
                                         2017: pd.Timestamp(('2017-06-28')),
                                         2018: pd.Timestamp(('2018-06-27')),
                                         2019: pd.Timestamp(('2019-06-26'))})
    for d in list(range(-4, 6)):
        df[f"wed_june{d}"] = (df.date - wed_june_date == np.timedelta64(d, "D")) & (df.country != 'Norway')
        
    # First Sunday of November - teckmengwong
    sun_nov_date = df.date.dt.year.map({2015: pd.Timestamp(('2015-11-1')),
                                         2016: pd.Timestamp(('2016-11-6')),
                                         2017: pd.Timestamp(('2017-11-5')),
                                         2018: pd.Timestamp(('2018-11-4')),
                                         2019: pd.Timestamp(('2019-11-3'))})
    df = pd.concat([df, pd.DataFrame({f"sun_nov{d}":
                                      (df.date - sun_nov_date == np.timedelta64(d, "D")) & (df.country == 'Norway')
                                      for d in list(range(0, 9))})], axis=1)
    
    # First half of December (Independence Day of Finland, 6th of December) -teckmengwong
    df = pd.concat([df, pd.DataFrame({f"dec{d}":
                                      (df.date.dt.month == 12) & (df.date.dt.day == d) & (df.country == 'Finland')
                                      for d in list(range(6, 14))})], axis=1)
    
    # Easter -teckmengwong
    easter_date = df.date.apply(lambda date: pd.Timestamp(easter.easter(date.year)))
    df = pd.concat([df, pd.DataFrame({f"easter{d}":
                                      (df.date - easter_date == np.timedelta64(d, "D"))
                                      for d in list(range(-2, 11)) + list(range(40, 48)) + list(range(50, 59))})], axis=1)
    
    return df

In [22]:
temporal_all_df = temporal_engineering(all_df)

In [23]:
temporal_all_df

,row_id,date,country,store,product,num_sold,gdp,month,season,wd4,...,easter47,easter50,easter51,easter52,easter53,easter54,easter55,easter56,easter57,easter58
0,0,2015-01-01,Finland,KaggleMart,Kaggle Mug,329.0,5.461456,1,1,False,...,False,False,False,False,False,False,False,False,False,False
1,1,2015-01-01,Finland,KaggleMart,Kaggle Hat,520.0,5.461456,1,1,False,...,False,False,False,False,False,False,False,False,False,False
2,2,2015-01-01,Finland,KaggleMart,Kaggle Sticker,146.0,5.461456,1,1,False,...,False,False,False,False,False,False,False,False,False,False
3,3,2015-01-01,Finland,KaggleRama,Kaggle Mug,572.0,5.461456,1,1,False,...,False,False,False,False,False,False,False,False,False,False
4,4,2015-01-01,Finland,KaggleRama,Kaggle Hat,911.0,5.461456,1,1,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6565,32863,2019-12-31,Sweden,KaggleMart,Kaggle Hat,NaN,6.282042,12,1,False,...,False,False,False,False,False,False,False,False,False,False
6566,32864,2019-12-31,Sweden,KaggleMart,Kaggle Sticker,NaN,6.282042,12,1,False,...,False,False,False,False,False,False,False,False,False,False
6567,32865,2019-12-31,Sweden,KaggleRama,Kaggle Mug,NaN,6.282042,12,1,False,...,False,False,False,False,False,False,False,False,False,False
6568,32866,2019-12-31,Sweden,KaggleRama,Kaggle Hat,NaN,6.282042,12,1,False,...,False,False,False,False,False,False,False,False,False,False


At this point, the `temporal_all_df` DataFrame contains all the time features for both the training and testing sets.
* **Todo**: consider not only adding in holidays from `holidays`, but also borrowing ideas from the AmbrosM Linear notebook too (which creates fewer features, populating them instead with temporal distances from the selected holidays).

### Target Transformation
Now, I'll do the target transformation proposed by @AmbrosM. (I'll do it to the non-encoded DataFrame too, for testing with Prophet and NeuralProphet later.)

In [24]:
for df in [temporal_all_df]:
    df['target'] = np.log(df['num_sold'] / df['gdp']**gdp_exponent)

In [25]:
# encoded_all_df['target'] = np.log(encoded_all_df['num_sold'] / (encoded_all_df['gdp']**gdp_exponent))

In [26]:
temporal_all_df

,row_id,date,country,store,product,num_sold,gdp,month,season,wd4,...,easter50,easter51,easter52,easter53,easter54,easter55,easter56,easter57,easter58,target
0,0,2015-01-01,Finland,KaggleMart,Kaggle Mug,329.0,5.461456,1,1,False,...,False,False,False,False,False,False,False,False,False,3.738239
1,1,2015-01-01,Finland,KaggleMart,Kaggle Hat,520.0,5.461456,1,1,False,...,False,False,False,False,False,False,False,False,False,4.196010
2,2,2015-01-01,Finland,KaggleMart,Kaggle Sticker,146.0,5.461456,1,1,False,...,False,False,False,False,False,False,False,False,False,2.925788
3,3,2015-01-01,Finland,KaggleRama,Kaggle Mug,572.0,5.461456,1,1,False,...,False,False,False,False,False,False,False,False,False,4.291321
4,4,2015-01-01,Finland,KaggleRama,Kaggle Hat,911.0,5.461456,1,1,False,...,False,False,False,False,False,False,False,False,False,4.756724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6565,32863,2019-12-31,Sweden,KaggleMart,Kaggle Hat,NaN,6.282042,12,1,False,...,False,False,False,False,False,False,False,False,False,NaN
6566,32864,2019-12-31,Sweden,KaggleMart,Kaggle Sticker,NaN,6.282042,12,1,False,...,False,False,False,False,False,False,False,False,False,NaN
6567,32865,2019-12-31,Sweden,KaggleRama,Kaggle Mug,NaN,6.282042,12,1,False,...,False,False,False,False,False,False,False,False,False,NaN
6568,32866,2019-12-31,Sweden,KaggleRama,Kaggle Hat,NaN,6.282042,12,1,False,...,False,False,False,False,False,False,False,False,False,NaN


### Label Encoding

I'm going to encapsulate this in a function so that it can be invoked just-in-time, in the hopes of avoiding confusions with DataFrames.

In [27]:
def label_encoder(df):
    from sklearn.preprocessing import LabelEncoder
    features = ['country', 'product', 'store']
    le_dict = {feature: LabelEncoder().fit(orig_train_df[feature]) for feature in features}
    enc_df = df.copy()
    for feature in features:
        enc_df[feature] = le_dict[feature].transform(df[feature])
    return le_dict, enc_df

In [28]:
# for key in le_dict.keys():
#     print(f"Values for key {key} are {le_dict[key].inverse_transform(range(len(le_dict[key].values())))}")#"
# print(le_dict['country'].inverse_transform([0,1,2]))
# print(le_dict['product'].inverse_transform([0,1,2]))
# print(le_dict['store'].inverse_transform([0,1]))

```
['Finland' 'Norway' 'Sweden']
['Kaggle Hat' 'Kaggle Mug' 'Kaggle Sticker']
['KaggleMart' 'KaggleRama']
```

Now, we'll do the encoding.

At this point, the `encoded_all_df` can be used -- perhaps with a call to `LabelEncoder.inverse_transform` -- to recover the "original" data when necessary (e.g. for feeding it into Prophet and NeuralProphet)

In [29]:
# encoded_all_df = label_encoder(temporal_all_df)

### Pseudolabeling

I'm not going to try this right now, but I may return to it later -- I note that Teck Meng Wong had some good results with it.

### Data Splitting, Modification

Now that the preprocessing is done, I'm going to split the data back into the train and test sets; then, I'll create a view on the dataframes that omits the year. The year-less dataframes will be suitable for residual learning.

In [30]:
# all_df = encoded_all_df.drop(columns=['num_sold', 'row_id'])
all_df = temporal_all_df.drop(columns=['row_id']) # writing over the previous version of `all_df`

In [31]:
tv_df = all_df[:len(orig_train_df)] # training and validation sets -- still not encoded
test_df = all_df[len(orig_train_df):] # still not encoded


In [32]:
# train_df = encoded_all_df.iloc[np.where(encoded_all_df['date'] < '2019-01-01'), :]
# test_df = encoded_all_df[[np.where(encoded_all_df['date'] > '2018-12-31')]]

# encoded_tv_df = encoded_all_df.drop(columns=['row_id'])[:len(orig_train_df)]
# encoded_test_df = encoded_all_df.drop(columns=['row_id'])[len(orig_train_df):]

# valid_df = tv_df[tv_df['date'] > '2017-12-31']
# train_df = tv_df[tv_df['date'] <= '2017-12-31']

# train_and_valid_residual_df = train_and_valid_df.drop(columns=['date'])
# test_residual_df = test_df.drop(columns=['date'])

# len(valid_df) + len(train_df) == len(tv_df)

# encoded_tv_df

# Training

### Forecasting Models Prep
First, we'll set up functions to handle the training of forecasting models which will discern trends, and which may -- or may not -- yield insights concerning seasonality. While the Scikit-Learn models will be able to share a single trainer function, the Prophet and NeuralProphet models have subtly different expectations of their data, and as such will require separate handling.

In [33]:
from sklearn.linear_model import Ridge, HuberRegressor, LinearRegression, Lasso
from sklearn.neural_network import MLPRegressor
from prophet import Prophet
from neuralprophet import NeuralProphet
# earth? wouldn't install via pip on my machine at first

In [34]:
import torch
import torch.nn as nn
from torch.nn.utils import weight_norm
from skorch import NeuralNetRegressor
import torch.optim as optim

#### (Preprepared Preds)

The next cell contains code to import already-existing predictions -- but I think it's better to centralize the code that produces them here, and will comment out the import code for now.

In [35]:
# prophet_trainset = load(predpath/'20220121_prophet_baseline_trainset.joblib')

# neural_trainset = load(predpath/'20220121_neuralprophet_baseline_trainset.joblib')
# neural_test_preds = load(predpath/'20220121_neuralprophet_baseline_testset.joblib')

# ridge_tv_preds = load(predpath/'20210121_ridge_baseline_trainset_preds.joblib')
# ridge_test_preds = load(predpath/'20220121_ridge_testset_preds.joblib')

And this cell would handle the parsing

In [36]:
# neural_tv_preds = neural_trainset['prophet_forecast']
# prophet_tv_preds = prophet_trainset['prophet_forecast']

# neural_train_preds = neural_tv_preds[:train_length]
# neural_valid_preds = neural_tv_preds[train_length:]

# prophet_train_preds = prophet_tv_preds[:train_length]
# prophet_valid_preds = prophet_tv_preds[train_length:]

# train_length = len(neural_trainset[neural_trainset['date'] <= '2017-12-31'])

# ridge_train_preds = ridge_tv_preds[:train_length]
# ridge_valid_preds = ridge_tv_preds[train_length:]

#### Scikit-Learn Linear Models Prep

Linear models from Scikit-Learn seemingly require that datetime data be converted to numerics.

In [37]:
# train_linear_df = train_df.copy()
# valid_linear_df = valid_df.copy()
# test_linear_df = test_df.copy()
# tv_linear_df = tv_df.copy()



### Forecasters

#### Hyperparameters
I'll hard-code them for now, but in the future may Optuna them. May want to create a dict of all the kwargs to be used for all the models, with the model names as keys

In [38]:
prophet_kwargs = {
    'growth':'linear',
#     'holidays':holidays_train, # will add this in-function
    'n_changepoints':10,
    'changepoint_range':0.4,
    'yearly_seasonality':True,
    'weekly_seasonality':True,
    'daily_seasonality':False,
    'seasonality_mode':'additive',
    'seasonality_prior_scale':25,
    'holidays_prior_scale':100,
    'changepoint_prior_scale':0.01,
    'interval_width':0.5,
    'uncertainty_samples':False
}

neuralprophet_kwargs = {
    'growth':'linear',
    'n_changepoints':10,
    'changepoints_range':0.4,
    'trend_reg':1,
    'trend_reg_threshold':False,
    'yearly_seasonality':True,
    'weekly_seasonality':True,
    'daily_seasonality':False,
    'seasonality_mode':'additive',
    'seasonality_reg':1,
    'n_forecasts':365,
    'normalize':'off'
}

# for pytorch / skorch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

tcn_kwargs = {
#     'module': estimator, # will be handled at-call
    'criterion': nn.MSELoss, # consider enhancement here
    "lr": 0.01, # default is 0.01
    'optimizer':Adam,
    'max_epochs':10, # default is 10
    'device': 'cpu'#device,
}

# model_params['hyperparams'] = str(neuralprophet_kwargs)
# model_params['holiday_source'] = 'Prophet builtin for each country'

In [39]:
best_xgboost_params = load(studypath/'optuna_xgboost_study-20220126213551.joblib').best_trial.params
best_xgboost_params['max_depth'] = best_xgboost_params['depth']
del best_xgboost_params['depth']
best_xgboost_params

{'n_estimators': 4207,
 'learning_rate': 0.05378597302351865,
 'reg_alpha': 0.0067949392113948815,
 'reg_lambda': 0.04865823628931899,
 'subsample': 0.212875760245356,
 'min_child_weight': 6.997692447967251,
 'colsample_bytree': 0.9824893256584818,
 'gamma': 0.10395228539921328,
 'max_depth': 5}

In [40]:
best_catboost_params = load(studypath/'optuna_catboost_study-20220127082356.joblib').best_trial.params
best_catboost_params['max_depth'] = best_catboost_params['depth']
del best_catboost_params['depth']
best_catboost_params

{'iterations': 10529,
 'learning_rate': 0.07026263205443048,
 'random_strength': 44,
 'od_wait': 261,
 'reg_lambda': 35.672029887566374,
 'border_count': 57,
 'min_child_samples': 19,
 'leaf_estimation_iterations': 2,
 'max_depth': 3}

In [41]:
best_lightgbm_params = load(studypath/'optuna_lightgbm_study-20220127171126.joblib').best_trial.params
best_lightgbm_params['max_depth'] = best_lightgbm_params['depth']
del best_lightgbm_params['depth']
best_lightgbm_params


{'n_estimators': 6078,
 'learning_rate': 0.03612108919426432,
 'reg_alpha': 0.008631524966022684,
 'reg_lambda': 0.19537138720003774,
 'subsample': 0.9601129223632775,
 'min_child_samples': 24,
 'num_leaves': 235,
 'colsample_bytree': 0.920126987868937,
 'max_depth': 3}

In [42]:
xgboost_params = {
    # universal
#     'tree_method': 'gpu_hist',
#     'predictor': 'gpu_predictor',
#     'eval_metric': ['mae', 'mape', 'rmse'],
#     'sampling_method': 'gradient_based',
#     'grow_policy': 'lossguide',
    
    # best of 500 trials on Optuna
    **best_xgboost_params
}


lightgbm_params = {
    # universal
    'objective': 'mse',
    'random_state': 42,
    'device_type': 'cpu',
    'n_jobs': -1,
#                 eval_metric='auc',
#     'device_type': 'gpu',
#     'max_bin': 63, # 15 might be even better for GPU perf, but depends on dataset -- see https://lightgbm.readthedocs.io/en/latest/GPU-Performance.html
#     'gpu_use_dp': False,
#     'max_depth': 0,
#     'learning_rate': 0.1,
#     'subsample': .15,
#     'n_estimators': 1500,
    **best_lightgbm_params
}

catboost_params = {
    # universal
#     'task_type':'GPU',
#     'silent':True,
#     'random_state':42,
    
    # from trial 4 (of 5) via Optuna
    **best_catboost_params
}
                

#### Temporal Convolutional Network

Implementation from https://www.kaggle.com/ceshine/pytorch-temporal-convolutional-networks.

In [43]:
class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        
        # this is the first convolutional layer; note that it foregoes padding irrespective of argument
        self.conv1 = weight_norm(nn.Conv2d(n_inputs, n_outputs, (1, kernel_size),
                                           stride=stride, padding=0, dilation=dilation))
        # the padding is then added after the first conv layer
        self.pad = torch.nn.ZeroPad2d((padding, 0, 0, 0))
        # this is a very standard choice
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        
        # the second convolutional layer in the block is identical to the first, but now padding has been added to the input
        self.conv2 = weight_norm(nn.Conv2d(n_outputs, n_outputs, (1, kernel_size),
                                           stride=stride, padding=0, dilation=dilation))
        
        # this simply strings together the above architectural elements, for convenience I guess
        self.net = nn.Sequential(self.pad, self.conv1, self.relu, self.dropout,
                                 self.pad, self.conv2, self.relu, self.dropout)
        
        # if the n_outputs is nonzero, this adds on a final convlutional layer to ensure that we get the desired number of outputs
        self.downsample = nn.Conv1d(
            n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        
        # this initializes the weights as specified in the separate weight initialization method, below
        self.init_weights()

    def init_weights(self):
        # this method initializes the weights for the Conv1D and Conv2D layers, plus the Downsample layer (if it's used)
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        # note the nice one-liner here, to add in the requisite number of dimensions both inbound to the NN and outbound
        out = self.net(x.unsqueeze(2)).squeeze(2) # original
#         out = self.net(x.unsqueeze(3)).squeeze(3) # my revision to address RuntimeError: Expected 4-dimensional input for 4-dimensional weight [32, 128, 1, 2], but got 3-dimensional input of size [128, 244, 2] instead
#         out = self.net(x.unsqueeze(3)).squeeze(2) # further revision to address IndexError: Dimension out of range (expected to be in range of [-3, 2], but got 3)
        # is this a residual, then?
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)

In [44]:
class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        '''
        What does num_channels mean? See Obsidian 202201270954... It seems that it should be a 
        list, with the number of hidden channels (i.e. activation units in each hidden layer), 
        repeated the number of hidden layers there are. E.g. [25,25,25,25]. An alternate idea:
        it's [hidden_size]*(level_size-1) + [embedding_size]
        
        I think that 
        '''
        
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

In [45]:
class TCNModel(nn.Module):
    def __init__(self, num_channels, kernel_size=2, dropout=0.2):
        super(TCNModel, self).__init__()
        self.tcn = TemporalConvNet(
            128, num_channels, kernel_size=kernel_size, dropout=dropout)
        self.dropout = nn.Dropout(dropout)
        self.decoder = nn.Linear(num_channels[-1], 1)

    def forward(self, x):
        return self.decoder(self.dropout(self.tcn(x)[:, :, -1]))

#### Trainers

##### NeuralProphet
I'm leaving the folds as they are. ~~Label encoding shouldn't matter -- the values are just being iterated over anyway.~~ It does matter because the Prophets use the strings to identify countries' holidays to add. Not sure about doing the target transform -- if you try it, just have the trainer call pass `target='target'`.

In [46]:
prophet_folds = [
    ('2015-01-01', '2018-01-01'),
    ('2018-01-01', '2019-01-01'),
]

In [47]:
# prophet_tv_df = tv_df_encoded.copy() # encoded_tv_df.copy()
# prophet_test_df = test_df_encoded.copy() # encoded_test_df.copy()

In [48]:
# for feature in ['country', 'product', 'store']:
#     prophet_tv_df[feature] = orig_train_df[feature]
#     prophet_test_df[feature] = orig_test_df[feature]


In [49]:
# prophet_tv_df.head()

In [50]:
# countries_enc = le_dict['country'].transform(countries)
# stores_enc = le_dict['store'].transform(stores)
# products_enc = le_dict['product'].transform(products)

# countries, countries_enc

In [51]:
def neuralprophet_trainer(model_kwargs=neuralprophet_kwargs, countries=countries, stores=stores, products=products, folds=prophet_folds, 
                          tv_df=tv_df, test_df=test_df,
#                           df_train=tv_df, df_test=test_df, 
                          target='num_sold', wandb_tracked=False):
    train_smape = 0
    val_smape = 0
    
    # create local versions of the dataframes, to avoid mutation
    df_train = tv_df.copy()
    df_test = test_df.copy()
    
    if wandb_tracked:
#         exmodel_config['arch'] = arch
#         exmodel_config[f'{arch}_params'] = str(model_params)
        wandb.init(
            project="202201_Kaggle_tabular_playground",
            save_code=True,
            tags=wandb_config['tags'],
            name=wandb_config['name'],
            notes=wandb_config['notes'],
            config=exmodel_config
    )
    
    # no label encoding here -- but test it with too
    for country in countries:
        for store in stores:
            for product in products:
                for fold, (start, end) in enumerate(folds):
                    # Skip iteration if it's the last fold
                    if fold == len(folds) - 1:
                        continue

                    # put only those rows in that are in the training window and have the correct country, store, and product
                    train_idx = (df_train['date'] >= start) &\
                                (df_train['date'] < end) &\
                                (df_train['country'] == country) &\
                                (df_train['store'] == store) &\
                                (df_train['product'] == product)

                    # redefine the training set in the local (holdout) sense
                    train = df_train.loc[train_idx, ['date', target]].reset_index(drop=True)

                    val_idx = (df_train['date'] >= folds[fold + 1][0]) &\
                              (df_train['date'] < folds[fold + 1][1]) &\
                              (df_train['country'] == country) &\
                              (df_train['store'] == store) &\
                              (df_train['product'] == product)

                    val = df_train.loc[val_idx, ['date', target]].reset_index(drop=True)

                    # rename the columns for standardization (this seems conventional)
                    train = train.rename(columns={'date': 'ds', target: 'y'})
                    val = val.rename(columns={'date': 'ds', target: 'y'})

#                     model = Prophet(**prophet_kwargs)
                    model = NeuralProphet(**model_kwargs)

                    model = model.add_country_holidays(country_name=country) # uses FacebookProphet or NeuralProphet API to add holidays
                    print(train.columns)
                    model.fit(train, freq='D') # neuralprophet
                    # prophet
#                     train_predictions = model.predict(train[['ds']])['yhat']
#                     val_predictions = model.predict(val[['ds']])['yhat']
                    # neuralprophet
                    train_predictions = model.predict(train)['yhat1']
                    val_predictions = model.predict(val)['yhat1']
                    df_train.loc[train_idx, 'neuralprophet_forecast'] = train_predictions.values
                    df_train.loc[val_idx, 'neuralprophet_forecast'] =  val_predictions.values

                    train_score = SMAPE(train['y'].values, train_predictions.values)
                    val_score = SMAPE(val['y'].values, val_predictions.values)
            
                    if wandb_tracked:
                        wandb.log({f"{(country,store,product)}_valid_smape": val_score})
            
                    train_smape += train_score
                    val_smape += val_score
            
                    print(f'\nTraining Range [{start}, {end}) - {country} - {store} - {product} - Train SMAPE: {train_score:4f}')
                    print(f'Validation Range [{folds[fold + 1][0]}, {folds[fold + 1][1]}) - {country} - {store} - {product} - Validation SMAPE: {val_score:4f}\n')

                    test_idx = (df_test['country'] == country) &\
                               (df_test['store'] == store) &\
                               (df_test['product'] == product)
                    test = df_test.loc[test_idx, ['date']].reset_index(drop=True)
                    
                    test = test.rename(columns={'date': 'ds'})
                    test['y'] = np.nan
                    test_predictions = model.predict(test)['yhat1']
                    
                    
                    df_test.loc[test_idx, 'neuralprophet_forecast'] = test_predictions.values
    
    train_smape /= (3*2*3)
    val_smape /= (3*2*3)
#     train_
    
    if wandb_tracked:
        wandb.log({'overall_train_smape': train_smape, 'overall_valid_smape': val_smape})
        wandb.finish()
    return df_train['neuralprophet_forecast'], df_test['neuralprophet_forecast']#, train_smape, val_smape

##### Prophet Trainer

In [52]:
def prophet_trainer(prophet_kwargs=prophet_kwargs, countries=countries, stores=stores, products=products, folds=prophet_folds, 
                    tv_df=tv_df, test_df=test_df,
#                           df_train=tv_df, df_test=test_df, 
                    target='num_sold', wandb_tracked=False):
    train_smape = 0
    val_smape = 0
    
    # create local versions of the dataframes, to avoid mutation
    df_train = tv_df.copy()
    df_test = test_df.copy()
    
    if wandb_tracked:
#         exmodel_config['arch'] = arch
#         exmodel_config[f'{arch}_params'] = str(model_params)
        wandb.init(
            project="202201_Kaggle_tabular_playground",
            save_code=True,
            tags=wandb_config['tags'],
            name=wandb_config['name'],
            notes=wandb_config['notes'],
            config=exmodel_config
    )
    
    for country in countries:
        for store in stores:
            for product in products:
                for fold, (start, end) in enumerate(folds):
                    # Skip iteration if it's the last fold
                    if fold == len(folds) - 1:
                        continue

                    # put only those rows in that are in the training window and have the correct country, store, and product
                    train_idx = (df_train['date'] >= start) &\
                                (df_train['date'] < end) &\
                                (df_train['country'] == country) &\
                                (df_train['store'] == store) &\
                                (df_train['product'] == product)
                    
#                     print(train_idx)

                    # redefine the training set in the local (holdout) sense
                    train = df_train.loc[train_idx, ['date', target]].reset_index(drop=True)
#                     print(train.shape)

                    val_idx = (df_train['date'] >= folds[fold + 1][0]) &\
                              (df_train['date'] < folds[fold + 1][1]) &\
                              (df_train['country'] == country) &\
                              (df_train['store'] == store) &\
                              (df_train['product'] == product)

                    val = df_train.loc[val_idx, ['date', target]].reset_index(drop=True)

                    # rename the columns for standardization (this seems conventional)
                    train = train.rename(columns={'date': 'ds', target: 'y'})
                    val = val.rename(columns={'date': 'ds', target: 'y'})

                    model = Prophet(**prophet_kwargs)

                    model.add_country_holidays(country_name=country) # uses FacebookProphet API to add holidays
                    model.fit(train)
        
                    train_predictions = model.predict(train[['ds']])['yhat']
                    val_predictions = model.predict(val[['ds']])['yhat']
                    df_train.loc[train_idx, 'prophet_forecast'] = train_predictions.values
                    df_train.loc[val_idx, 'prophet_forecast'] =  val_predictions.values

                    train_score = SMAPE(train['y'].values, train_predictions.values)
                    val_score = SMAPE(val['y'].values, val_predictions.values)
            
                    if wandb_tracked:
                        wandb.log({f"{(country,store,product)}_valid_smape": val_score})
            
                    train_smape += train_score
                    val_smape += val_score
            
                    print(f'\nTraining Range [{start}, {end}) - {country} - {store} - {product} - Train SMAPE: {train_score:4f}')
                    print(f'Validation Range [{folds[fold + 1][0]}, {folds[fold + 1][1]}) - {country} - {store} - {product} - Validation SMAPE: {val_score:4f}\n')

                    test_idx = (df_test['country'] == country) &\
                               (df_test['store'] == store) &\
                               (df_test['product'] == product)
                    test = df_test.loc[test_idx, ['date']].reset_index(drop=True)
                    
                    test = test.rename(columns={'date': 'ds'})
                    test_predictions = model.predict(test[['ds']])['yhat']
                    
                    
                    df_test.loc[test_idx, 'prophet_forecast'] = test_predictions.values
    
    train_smape /= (3*2*3)
    val_smape /= (3*2*3)
    
    if wandb_tracked:
        wandb.log({'overall_train_smape': train_smape, 'overall_valid_smape': val_smape})
        wandb.finish()
    return df_train['prophet_forecast'], df_test['prophet_forecast']#, train_smape, val_smape

##### Scikit-Learn Models

In [53]:
def sklearn_trainer(estimator, model_kwargs={}, tv_df=tv_df, test_df=test_df, #X=X, y=y, X_valid=X_valid, y_valid=y_valid, X_test=X_test, 
                    folds=prophet_folds, countries=countries, stores=stores, products=products, target='target',
#                     by_combo=True, 
                    model_type=None, # None -> fully scikit-learn compatible; alternatives are 'skorch' or 'gbm'
                    wandb_tracked=False):
    
    # create local versions of the dataframes, to avoid mutation
    df_train = tv_df.copy()
    df_test = test_df.copy()
    
    # apply label encoding (which Scikit-Learn models require, but *Prophets don't)
    le_dict, tv_df = label_encoder(df_train) # should leave broader scope's tv_df alone
    _, test_df = label_encoder(df_test) # should leave broader scope's test_df alone
    del df_train, df_test
    
    # encode the lists of countries, stores, and products
    countries = le_dict['country'].transform(countries)
    stores = le_dict['store'].transform(stores)
    products = le_dict['product'].transform(products)
    
    train_smape = 0
    val_smape = 0
    
    if wandb_tracked:
#         exmodel_config['arch'] = arch
#         exmodel_config[f'{arch}_params'] = str(model_params)
        wandb.init(
            project="202201_Kaggle_tabular_playground",
            save_code=True,
            tags=wandb_config['tags'],
            name=wandb_config['name'],
            notes=wandb_config['notes'],
            config=exmodel_config
    )
    
    # drop whichever version of the dependent variable is not being used
#     for df in [tv_df, test_df]:
    if target == 'num_sold': 
        tv_df = tv_df.drop(columns=['target'])
        test_df = test_df.drop(columns=['target'])
    else:
        tv_df = tv_df.drop(columns=['num_sold'])
        test_df = test_df.drop(columns=['num_sold'])
            
#     print("'num_sold' in test_df.columns == ", 'num_sold' in test_df.columns)
    
    # handling each combination of country, store, and product separately
    for country in countries:
        for store in stores:
            for product in products:
                for fold, (start, end) in enumerate(folds):
                    # Skip iteration if it's the last fold
                    if fold == len(folds) - 1:
                        continue

                    # put only those rows in that are in the training window and have the correct country, store, and product
                    train_idx = (tv_df['date'] >= start) &\
                                (tv_df['date'] < end) &\
                                (tv_df['country'] == country) &\
                                (tv_df['store'] == store) &\
                                (tv_df['product'] == product)

#                     print(train_idx)

                    # redefine the training set in the local (holdout) sense
                    train = tv_df.loc[train_idx, :].reset_index(drop=True)
#                         print(train.shape)

                    val_idx = (tv_df['date'] >= folds[fold + 1][0]) &\
                              (tv_df['date'] < folds[fold + 1][1]) &\
                              (tv_df['country'] == country) &\
                              (tv_df['store'] == store) &\
                              (tv_df['product'] == product)

                    val = tv_df.loc[val_idx, :].reset_index(drop=True)

                    test_idx = (test_df['country'] == country) &\
                               (test_df['store'] == store) &\
                               (test_df['product'] == product)
                    test = test_df.loc[test_idx, :].reset_index(drop=True)

                    # with the training and validation sets sorted out, make them integers for model fitting
                    for df in [train, val, test]:
                        df['date'] = df['date'].map(dt.datetime.toordinal)
                    if 'model_forecast' in train.columns:
                        X = train.drop(columns=[target, 'model_forecast'])
                        X_valid = val.drop(columns=[target, 'model_forecast'])
                        X_test = test.drop(columns=[target, 'model_forecast'])
                    else:
                        X = train.drop(columns=[target])
                        X_valid = val.drop(columns=[target])
                        X_test = test.drop(columns=[target])

                    y = train[target]
                    y_valid = val[target]


#                         print(type(X), type(y))
#                         print(f"X has {X.isna().any().sum()} NaNs")
#                         print(f"y has {y.isna().sum()} NaNs")
#                     print(X_test.info())
#                     print(y_valid.dtype)
    
                    if model_type == 'skorch':
#                         for df in [X, X_valid, X_test]:
# #                             df['date'] = df['date'].apply(dt.datetime.toordinal)
#                             df = torch.tensor(df.to_numpy(dtype=np.float32))
#                         for target in [y, y_valid]:
#                             target = torch.tensor(np.array(target))
# #                             target = target.reshape(-1,1)
#                             target = target.unsqueeze(0)
                        X = torch.tensor(X.to_numpy(dtype=np.float32))
                        X_valid = torch.tensor(X_valid.to_numpy(dtype=np.float32))
                        X_test = torch.tensor(X_test.to_numpy(dtype=np.float32))
            
                        y = torch.tensor(np.array(y)).reshape(-1,1)
                        y_valid = torch.tensor(np.array(y)).reshape(-1,1)
    
                        tcn_kwargs = {
                            'num_channels': [32,32,32,32],
                        }
                        print(type(y), type(y_valid))
#                         y = y.reshape(-1,1)
#                         y_valid = y_valid.reshape(-1,1)
                        # create the Datasets
                
                        # create the DataLoaders

                        # instantiate the wrapper
                        model = NeuralNetRegressor(
                            module=estimator(**tcn_kwargs),
                            **model_kwargs
                        )
#                     elif model_type=='gbm':
                        
                    else:
                        model = estimator(**model_kwargs)

                    model.fit(X,y)

                    model_train_preds = model.predict(X)
                    model_valid_preds = model.predict(X_valid)
                    model_test_preds = model.predict(X_test)

                    tv_df.loc[train_idx, 'model_forecast'] = model_train_preds#.values
                    tv_df.loc[val_idx, 'model_forecast'] =  model_valid_preds#.values
                    test_df.loc[test_idx, 'model_forecast'] = model_test_preds#.values


    # reverse the dependent variable transform if appropriate
    if target == 'target':
#             model_tv_preds = np.multiply(np.exp(model_tv_preds), tv_df['gdp']**gdp_exponent)
        tv_df['model_forecast'] = np.exp(tv_df['model_forecast']) * tv_df['gdp']**gdp_exponent
#             output_tv_df['model_forecast'] = np.exp(output_tv_df['model_forecast']) * output_tv_df['gdp']**gdp_exponent

#             model_test_preds = np.multiply(np.exp(model_test_preds), test_df['gdp']**gdp_exponent)
        test_df['model_forecast'] = np.exp(test_df['model_forecast']) * test_df['gdp']**gdp_exponent
#             output_test_df['model_forecast'] = np.exp(output_test_df['model_forecast']) * output_test_df['gdp']**gdp_exponent
#             model_test_preds = np.exp(model_test_preds) * test_df['gdp']**gdp_exponent
        
#         tv_df['model_forecast'] = model_tv_preds
#         test_df['model_forecast'] = model_test_preds
#     return output_tv_df, output_test_df
    return tv_df['model_forecast'], test_df['model_forecast']
#     return tv_df['model_forecast'], test_df['model_forecast']
#     return model_tv_preds, model_test_preds
    

##### GBMs

In [54]:
from sklearn.model_selection import GroupKFold

In [55]:
def gbm_trainer(arch:str, model_kwargs={}, tv_df=tv_df, test_df=test_df, #X=X, y=y, X_valid=X_valid, y_valid=y_valid, X_test=X_test, 
                countries=countries, stores=stores, products=products, 
                target='target', wandb_tracked=False):
    
    # create local versions of the dataframes, to avoid mutation
    X = tv_df.copy()
    X_test = test_df.copy()
    
    # apply label encoding (which Scikit-Learn models require, but *Prophets don't)
    le_dict, X = label_encoder(X) # should leave broader scope's tv_df alone
    _, X_test = label_encoder(X_test) # should leave broader scope's test_df alone
#     del df_train, df_test
    
    # encode the lists of countries, stores, and products
    countries = le_dict['country'].transform(countries)
    stores = le_dict['store'].transform(stores)
    products = le_dict['product'].transform(products)
    
#     train_smape = 0
#     val_smape = 0
    
    if wandb_tracked:
#         exmodel_config['arch'] = arch
#         exmodel_config[f'{arch}_params'] = str(model_params)
        wandb.init(
            project="202201_Kaggle_tabular_playground",
            save_code=True,
            tags=wandb_config['tags'],
            name=wandb_config['name'],
            notes=wandb_config['notes'],
            config=exmodel_config
        )
    
    # drop whichever version of the dependent variable is not being used
#     for df in [tv_df, test_df]:
    y = X[target]
#     for df in [X, X_test]:
#         df = df.drop(columns=['num_sold', 'target'])
    X = X.drop(columns=['num_sold', 'target'])
    X_test = X_test.drop(columns=['num_sold', 'target'])
#     X = X.drop(columns)
#     if target == 'num_sold': 
#         y = X['num_sold']
#         X = X.drop(columns=['target'])
#         X_test = X_test.drop(columns=['target'])
#     else:
#         X = X.drop(columns=['num_sold'])
#         X_test = X_test.drop(columns=['num_sold'])
    
    kfold = GroupKFold(n_splits=4)
    oof_preds = pd.Series(0, index=tv_df.index)
#     oof_preds, oof_y = [], []
#     test_preds = np.zeros((X_test.shape[0]))
    test_preds = pd.Series(0, index=test_df.index)
    
    for fold, (train_ids, valid_ids) in enumerate(kfold.split(tv_df, groups=tv_df.date.dt.year)):
        print(f"FOLD {fold}")
        print("------------------------------")
        
        # remove dates 
#         for df in [X, X_test]:
#             df = df.drop(columns=['date'])
        if 'date' in X.columns:
            X = X.drop(columns=['date'])
            X_test = X_test.drop(columns=['date'])#, 'num_sold'])
        
        y_train, y_valid = y[train_ids], y[valid_ids]
        X_train, X_valid = X.iloc[train_ids,:], X.iloc[valid_ids,:]
        
        if arch == 'xgboost':
            model = XGBRegressor(
                tree_method= 'gpu_hist',
                predictor= 'gpu_predictor',
                eval_metric= ['mae', 'mape'],
                sampling_method= 'gradient_based',
                grow_policy= 'lossguide',
                seed=42,
                objective='reg:squarederror',
                **model_kwargs)
            if wandb_tracked:
                model.fit(X_train, y_train, callbacks=[wandb.xgboost.wandb_callback()])
            else:
                model.fit(X_train, y_train)
        elif arch == 'lightgbm':
            model = LGBMRegressor(
                **model_kwargs)
            if wandb_tracked:
                model.fit(X_train, y_train, callbacks=[wandb.lightgbm.wandb_callback()])
            else:
                model.fit(X_train, y_train)
        
        elif arch == 'catboost':
            model = CatBoostRegressor(
                task_type='GPU',
                silent=True,
                random_state=42,
                **model_kwargs)
            model.fit(X_train, y_train)
        
        y_valid_preds = model.predict(X_valid)
        
        oof_preds[valid_ids] = y_valid_preds
#         oof_preds.extend(y_valid_preds)
#         oof_y.extend(y_valid)
        
        if arch == 'catboost':
            test_preds += model.predict(X_test).flatten()
        else:
            test_preds += model.predict(X_test)
        
#         fold_smape = SMAPE(y_true=y_valid, y_pred=y_valid_preds)
#         print(f"FOLD {fold} OOF SMAPE: {fold_smape}")
    test_preds /= 4 # taking the average of the test preds
    
    if target == 'target':
        oof_preds = np.exp(oof_preds) * tv_df['gdp']**gdp_exponent
        test_preds = np.exp(test_preds) * test_df['gdp']**gdp_exponent
        
    smape = SMAPE(y_pred=oof_preds, y_true=tv_df['num_sold'])
#     print("Lengths of oof_preds and tv_df[target] are same? ", len(oof_preds) == len(tv_df[target]))
#     print(oof_preds[:10])
#     print(tv_df[target][:10])
    print(f"SMAPE: {smape}")
    if wandb_tracked:
        wandb.log({
            'arch': arch,
            'SMAPE': smape,
            'model_params': str(model_kwargs),
            'model_seed': random_state
        })
        wandb.finish()
    return oof_preds, test_preds

#### Calls

In [56]:
%%time 
# xgb_forecast_oof_preds, xgb_forecast_test_preds = gbm_trainer(arch='xgboost', model_kwargs=xgboost_params)
# dump(xgb_forecast_oof_preds, predpath/'20220127_xgboost_oof_preds.joblib')
# dump(xgb_forecast_test_preds, predpath/'20220127_xgboost_test_preds.joblib')
xgb_forecast_oof_preds = load(predpath/'20220127_xgboost_oof_preds.joblib')
xgb_forecast_test_preds = load(predpath/'20220127_xgboost_test_preds.joblib')

CPU times: user 1.88 ms, sys: 130 µs, total: 2.01 ms
Wall time: 2.04 ms


In [57]:
%%time 
# lgb_forecast_oof_preds, lgb_forecast_test_preds = gbm_trainer(arch='lightgbm', model_kwargs=lightgbm_params)
# dump(lgb_forecast_oof_preds, predpath/'20220127_lightgbm_oof_preds.joblib')
# dump(lgb_forecast_test_preds, predpath/'20220127_lightgbm_test_preds.joblib')
lgb_forecast_oof_preds = load(predpath/'20220127_lightgbm_oof_preds.joblib')
lgb_forecast_test_preds = load(predpath/'20220127_lightgbm_test_preds.joblib')

CPU times: user 2.15 ms, sys: 0 ns, total: 2.15 ms
Wall time: 1.95 ms


In [58]:
%%time 
# cat_forecast_oof_preds, cat_forecast_test_preds = gbm_trainer(arch='catboost', model_kwargs=catboost_params)
# dump(cat_forecast_oof_preds, predpath/'20220127_catboost_oof_preds.joblib')
# dump(cat_forecast_test_preds, predpath/'20220127_catboost_test_preds.joblib')

cat_forecast_oof_preds = load(predpath/'20220127_catboost_oof_preds.joblib')
cat_forecast_test_preds = load(predpath/'20220127_catboost_test_preds.joblib')

CPU times: user 1.82 ms, sys: 0 ns, total: 1.82 ms
Wall time: 1.79 ms


In [59]:
# %%time 
# tcn_tv_preds, tcn_test_preds = sklearn_trainer(estimator=TCNModel, model_type='skorch')#use_skorch=True)

In [60]:
%time
# prophet_tv_preds, prophet_test_preds = prophet_trainer(target='num_sold')
# dump(prophet_tv_preds, predpath/'20220127_prophet_oof_preds.joblib')
# dump(prophet_test_preds, predpath/'20220127_prophet_test_preds.joblib')

prophet_tv_preds = load(predpath/'20220127_prophet_oof_preds.joblib')
prophet_test_preds = load(predpath/'20220127_prophet_test_preds.joblib')

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.81 µs


In [61]:
%time
# neural_tv_preds, neural_test_preds = neuralprophet_trainer(target='num_sold')
# dump(neural_tv_preds, predpath/'20220124_neuralprophet_tv_preds.joblib')
# dump(neural_test_preds, predpath/'20220124_neuralprophet_test_preds.joblib')
neural_tv_preds = load(predpath/'20220124_neuralprophet_tv_preds.joblib')
neural_test_preds = load(predpath/'20220124_neuralprophet_test_preds.joblib')

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.34 µs


In [62]:
# %time
# ridge_tv_preds, ridge_test_preds = sklearn_trainer(estimator=Ridge)

In [63]:
%%time
ridge_combo_tv_preds, ridge_combo_test_preds = sklearn_trainer(estimator=Ridge)#, by_combo=True)

CPU times: user 6.35 s, sys: 142 ms, total: 6.49 s
Wall time: 865 ms


In [64]:
# linear_tv_preds, linear_test_preds = sklearn_trainer(estimator=LinearRegression)

In [65]:
%%time
linear_combo_tv_preds, linear_combo_test_preds = sklearn_trainer(estimator=LinearRegression)#, by_combo=True)

CPU times: user 7.45 s, sys: 139 ms, total: 7.59 s
Wall time: 949 ms


In [66]:
%%time
huber_combo_tv_preds, huber_combo_test_preds = sklearn_trainer(estimator=HuberRegressor)#, by_combo=True)

CPU times: user 2.22 s, sys: 39.6 ms, total: 2.26 s
Wall time: 1.04 s


In [67]:
%%time
mlp_combo_tv_preds, mlp_combo_test_preds = sklearn_trainer(estimator=MLPRegressor)#, by_combo=True)

CPU times: user 46.9 s, sys: 675 ms, total: 47.6 s
Wall time: 6.01 s


In [68]:
%%time
lasso_combo_tv_preds, lasso_combo_test_preds = sklearn_trainer(estimator=Lasso)#), by_combo=True)

CPU times: user 2.13 s, sys: 64.3 ms, total: 2.19 s
Wall time: 841 ms


In [69]:
from pyearth import Earth

In [70]:
%%time
# earth_combo_tv_preds, earth_combo_test_preds = sklearn_trainer(estimator=Earth), #by_combo=True)
# mystery = sklearn_trainer(estimator=Earth)
# dump(earth_combo_tv_preds, predpath/'20220127_earth_oof_preds.joblib')
# dump(earth_combo_test_preds, predpath/'20220127_earth_test_preds.joblib')

earth_combo_tv_preds = load(predpath/'20220127_earth_oof_preds.joblib')
earth_combo_test_preds = load(predpath/'20220127_earth_test_preds.joblib')

CPU times: user 0 ns, sys: 4.08 ms, total: 4.08 ms
Wall time: 3.56 ms


In [71]:
earth_combo_tv_preds

0         360.672190
1         584.851806
2         138.401176
3         562.479785
4         944.584508
            ...     
26293     836.947940
26294     254.287204
26295    1058.174059
26296    2036.549357
26297     472.307718
Name: model_forecast, Length: 26298, dtype: float64

In [72]:
y_tv = orig_train_df['num_sold']

In [73]:
for preds in [prophet_tv_preds, ridge_combo_tv_preds, linear_combo_tv_preds, huber_combo_tv_preds, mlp_combo_tv_preds, neural_tv_preds, lasso_combo_tv_preds, earth_combo_tv_preds, xgb_forecast_oof_preds, lgb_forecast_oof_preds, cat_forecast_oof_preds]:
    print(SMAPE(y_pred=preds, y_true=y_tv))

7.098824805150043
4.781041939812455
4.175573461450916
14.396222549610735
99.04042806269284
7.926662675729486
15.11713989060885
5.338351055481194
6.188333122674364
6.045153269931954
5.952674821230676


Clearly, doing it by combination is the way to go. I think Ridge and LinearRegressor are definitely good to use; Prophet and NeuralProphet are worth including as well. Huber, probably not; MLPRegressor, definitely not barring the discovery of better hyperparams.

### Forecast Bundling
Now, create an iterable collection consisting of all the forecaster predictions.

In [74]:
forecast_tv_preds = {
    'prophet': prophet_tv_preds,
    'neuralprophet': neural_tv_preds,
    'ridge': ridge_combo_tv_preds,
    'linear': linear_combo_tv_preds,
#     'huber': huber_combo_tv_preds,
#     'lasso': lasso_combo_tv_preds,
    'earth': earth_combo_tv_preds,
    'xgboost': xgb_forecast_oof_preds, 
    'lightgbm': lgb_forecast_oof_preds, 
    'catboost': cat_forecast_oof_preds
}

forecast_test_preds = {
    'prophet': prophet_test_preds,
    'neuralprophet': neural_test_preds,
    'ridge': ridge_combo_test_preds,
    'linear': linear_combo_test_preds,
#     'huber': huber_combo_test_preds,
#     'lasso': lasso_combo_test_preds,
    'earth': earth_combo_test_preds,
    'xgboost': xgb_forecast_test_preds, 
    'lightgbm': lgb_forecast_test_preds, 
    'catboost': cat_forecast_test_preds

}

In [75]:
tv_preds = pd.DataFrame({
    'date': orig_train_df['date'],
    'num_sold': orig_train_df['num_sold'],
    **forecast_tv_preds
})

In [76]:
tv_preds

,date,num_sold,prophet,neuralprophet,ridge,linear,earth,xgboost,lightgbm,catboost
0,2015-01-01,329,346.560416,329.134521,283.272258,324.188332,360.672190,343.250478,331.496226,326.054356
1,2015-01-01,520,536.203586,458.008301,439.771734,503.081569,584.851806,510.616623,514.576867,509.005936
2,2015-01-01,146,143.412803,145.325577,120.885683,136.667932,138.401176,148.959080,146.429925,144.894609
3,2015-01-01,572,590.117165,552.571167,488.810710,555.531957,562.479785,596.463984,584.653899,569.038022
4,2015-01-01,911,939.673009,781.967285,771.701702,890.469201,944.584508,907.745110,888.842963,882.892654
...,...,...,...,...,...,...,...,...,...,...
26293,2018-12-31,823,898.322121,669.418457,723.714830,846.597603,836.947940,830.633157,814.597001,825.377960
26294,2018-12-31,250,253.512355,227.158142,205.880079,241.048951,254.287204,241.115195,234.409908,245.208674
26295,2018-12-31,1004,1039.635205,715.639648,832.192362,975.785339,1058.174059,898.057926,900.745370,915.158956
26296,2018-12-31,1441,1526.908216,980.234009,1255.885410,1468.776593,2036.549357,1397.507711,1419.978895,1401.414363


In [77]:
valid_forecast_preds = tv_preds[tv_preds['date'] > '2017-12-31']
valid_forecast_preds

,date,num_sold,prophet,neuralprophet,ridge,linear,earth,xgboost,lightgbm,catboost
19728,2018-01-01,405,383.365770,374.809906,345.296498,417.388135,382.162985,353.056594,343.574789,361.219774
19729,2018-01-01,621,602.594228,539.082520,536.708119,637.949409,609.231013,566.945177,532.836041,561.683760
19730,2018-01-01,176,161.735807,167.919098,147.203184,174.086997,167.839584,152.021516,151.443938,158.623556
19731,2018-01-01,714,656.201669,635.251648,596.599377,713.788713,597.369896,630.848983,594.969939,635.689518
19732,2018-01-01,1043,1044.903702,940.582886,930.576635,1110.805235,1059.300177,958.739937,917.331354,981.698729
...,...,...,...,...,...,...,...,...,...,...
26293,2018-12-31,823,898.322121,669.418457,723.714830,846.597603,836.947940,830.633157,814.597001,825.377960
26294,2018-12-31,250,253.512355,227.158142,205.880079,241.048951,254.287204,241.115195,234.409908,245.208674
26295,2018-12-31,1004,1039.635205,715.639648,832.192362,975.785339,1058.174059,898.057926,900.745370,915.158956
26296,2018-12-31,1441,1526.908216,980.234009,1255.885410,1468.776593,2036.549357,1397.507711,1419.978895,1401.414363


In [78]:
valid_forecast_preds.corr()

,num_sold,prophet,neuralprophet,ridge,linear,earth,xgboost,lightgbm,catboost
num_sold,1.000000,0.972938,0.961440,0.992806,0.995334,0.984322,0.993704,0.993255,0.994466
prophet,0.972938,1.000000,0.990336,0.985617,0.975158,0.972174,0.979772,0.981227,0.976901
neuralprophet,0.961440,0.990336,1.000000,0.978596,0.962073,0.961384,0.968638,0.971006,0.965273
ridge,0.992806,0.985617,0.978596,1.000000,0.996412,0.989698,0.997007,0.997152,0.996544
linear,0.995334,0.975158,0.962073,0.996412,1.000000,0.987908,0.996964,0.996341,0.997524
earth,0.984322,0.972174,0.961384,0.989698,0.987908,1.000000,0.991375,0.991292,0.991564
xgboost,0.993704,0.979772,0.968638,0.997007,0.996964,0.991375,1.000000,0.999264,0.999068
lightgbm,0.993255,0.981227,0.971006,0.997152,0.996341,0.991292,0.999264,1.000000,0.998589
catboost,0.994466,0.976901,0.965273,0.996544,0.997524,0.991564,0.999068,0.998589,1.000000


In [79]:
tv_preds.corr()

,num_sold,prophet,neuralprophet,ridge,linear,earth,xgboost,lightgbm,catboost
num_sold,1.000000,0.975093,0.960082,0.993110,0.995846,0.990016,0.991070,0.990342,0.991177
prophet,0.975093,1.000000,0.985693,0.986481,0.977785,0.974833,0.980087,0.979258,0.977476
neuralprophet,0.960082,0.985693,1.000000,0.978010,0.962507,0.960241,0.966489,0.964066,0.961882
ridge,0.993110,0.986481,0.978010,1.000000,0.996573,0.992161,0.994821,0.993991,0.994014
linear,0.995846,0.977785,0.962507,0.996573,1.000000,0.994045,0.994207,0.993206,0.994058
earth,0.990016,0.974833,0.960241,0.992161,0.994045,1.000000,0.988929,0.987730,0.988268
xgboost,0.991070,0.980087,0.966489,0.994821,0.994207,0.988929,1.000000,0.997599,0.997227
lightgbm,0.990342,0.979258,0.964066,0.993991,0.993206,0.987730,0.997599,1.000000,0.998696
catboost,0.991177,0.977476,0.961882,0.994014,0.994058,0.988268,0.997227,0.998696,1.000000


So Ridge and Linear perform best, and are quite similar; Prophet and NeuralProphet are next best, and similar to one another; Huber is an outlier (and not so good). Lasso performs worst of all, but is closer to Prophet and NeuralProphet in performance than the others. It's closest of all to NeuralProphet, interestingly.

In [80]:
test_forecast_preds = pd.DataFrame({
    'date': orig_test_df['date'],
#     'num_sold': orig_train_df['num_sold'],
    **forecast_test_preds
})

In [81]:
test_forecast_preds

,date,prophet,neuralprophet,ridge,linear,earth,xgboost,lightgbm,catboost
0,2019-01-01,402.296576,383.543457,359.817081,392.328390,417.102143,349.759459,373.034564,379.702323
1,2019-01-01,631.642099,551.044983,560.783324,620.092467,693.844970,549.911123,578.563016,589.688285
2,2019-01-01,169.992094,173.065674,153.552940,166.788184,181.642807,152.988182,165.419045,168.116727
3,2019-01-01,684.559821,648.783691,622.237644,675.674104,594.166542,622.932486,651.424049,668.232707
4,2019-01-01,1092.875362,965.683716,969.065554,1084.980801,1142.564632,943.483740,994.810045,1033.224703
...,...,...,...,...,...,...,...,...,...
6565,2019-12-31,917.085861,674.891174,742.331990,797.964290,830.606248,780.494827,809.243998,804.655222
6566,2019-12-31,261.569590,232.082169,210.920805,225.576610,269.498973,229.878546,235.835586,239.325998
6567,2019-12-31,1065.400121,730.457275,852.091174,925.788409,1133.560974,872.192736,899.608764,895.495983
6568,2019-12-31,1568.407214,1005.392944,1286.814471,1397.071983,2627.470943,1358.919453,1380.445879,1356.416644


## Submission of Naive Forecasts

Something's up with the residuals; for now, let's just run a simple model on the original forecasts.

In [82]:
model = Lasso()
# model = Ridge() # not good
# model = LinearRegression() # not good
# model = CatBoostRegressor(**catboost_params) # very bad

In [83]:
tv_preds

,date,num_sold,prophet,neuralprophet,ridge,linear,earth,xgboost,lightgbm,catboost
0,2015-01-01,329,346.560416,329.134521,283.272258,324.188332,360.672190,343.250478,331.496226,326.054356
1,2015-01-01,520,536.203586,458.008301,439.771734,503.081569,584.851806,510.616623,514.576867,509.005936
2,2015-01-01,146,143.412803,145.325577,120.885683,136.667932,138.401176,148.959080,146.429925,144.894609
3,2015-01-01,572,590.117165,552.571167,488.810710,555.531957,562.479785,596.463984,584.653899,569.038022
4,2015-01-01,911,939.673009,781.967285,771.701702,890.469201,944.584508,907.745110,888.842963,882.892654
...,...,...,...,...,...,...,...,...,...,...
26293,2018-12-31,823,898.322121,669.418457,723.714830,846.597603,836.947940,830.633157,814.597001,825.377960
26294,2018-12-31,250,253.512355,227.158142,205.880079,241.048951,254.287204,241.115195,234.409908,245.208674
26295,2018-12-31,1004,1039.635205,715.639648,832.192362,975.785339,1058.174059,898.057926,900.745370,915.158956
26296,2018-12-31,1441,1526.908216,980.234009,1255.885410,1468.776593,2036.549357,1397.507711,1419.978895,1401.414363


In [84]:
X = tv_preds.drop(columns=['num_sold'])
y = tv_preds['num_sold']

In [85]:
test_forecast_preds

,date,prophet,neuralprophet,ridge,linear,earth,xgboost,lightgbm,catboost
0,2019-01-01,402.296576,383.543457,359.817081,392.328390,417.102143,349.759459,373.034564,379.702323
1,2019-01-01,631.642099,551.044983,560.783324,620.092467,693.844970,549.911123,578.563016,589.688285
2,2019-01-01,169.992094,173.065674,153.552940,166.788184,181.642807,152.988182,165.419045,168.116727
3,2019-01-01,684.559821,648.783691,622.237644,675.674104,594.166542,622.932486,651.424049,668.232707
4,2019-01-01,1092.875362,965.683716,969.065554,1084.980801,1142.564632,943.483740,994.810045,1033.224703
...,...,...,...,...,...,...,...,...,...
6565,2019-12-31,917.085861,674.891174,742.331990,797.964290,830.606248,780.494827,809.243998,804.655222
6566,2019-12-31,261.569590,232.082169,210.920805,225.576610,269.498973,229.878546,235.835586,239.325998
6567,2019-12-31,1065.400121,730.457275,852.091174,925.788409,1133.560974,872.192736,899.608764,895.495983
6568,2019-12-31,1568.407214,1005.392944,1286.814471,1397.071983,2627.470943,1358.919453,1380.445879,1356.416644


In [86]:
X_test = test_forecast_preds

In [87]:
X['date'] = pd.to_datetime(X.date)

In [88]:
X['date'] = X['date'].map(dt.datetime.toordinal)

In [89]:
# X = X.drop(columns=['huber'])

In [90]:
model.fit(X,y)

Lasso()

In [91]:
# abs(model.coef_)

In [92]:
X_test

,date,prophet,neuralprophet,ridge,linear,earth,xgboost,lightgbm,catboost
0,2019-01-01,402.296576,383.543457,359.817081,392.328390,417.102143,349.759459,373.034564,379.702323
1,2019-01-01,631.642099,551.044983,560.783324,620.092467,693.844970,549.911123,578.563016,589.688285
2,2019-01-01,169.992094,173.065674,153.552940,166.788184,181.642807,152.988182,165.419045,168.116727
3,2019-01-01,684.559821,648.783691,622.237644,675.674104,594.166542,622.932486,651.424049,668.232707
4,2019-01-01,1092.875362,965.683716,969.065554,1084.980801,1142.564632,943.483740,994.810045,1033.224703
...,...,...,...,...,...,...,...,...,...
6565,2019-12-31,917.085861,674.891174,742.331990,797.964290,830.606248,780.494827,809.243998,804.655222
6566,2019-12-31,261.569590,232.082169,210.920805,225.576610,269.498973,229.878546,235.835586,239.325998
6567,2019-12-31,1065.400121,730.457275,852.091174,925.788409,1133.560974,872.192736,899.608764,895.495983
6568,2019-12-31,1568.407214,1005.392944,1286.814471,1397.071983,2627.470943,1358.919453,1380.445879,1356.416644


In [93]:
# X_test['date'] = X['date'].map(dt.datetime.toordinal)
X_test['date'] = pd.to_datetime(X_test.date)

In [94]:
X_test

,date,prophet,neuralprophet,ridge,linear,earth,xgboost,lightgbm,catboost
0,2019-01-01,402.296576,383.543457,359.817081,392.328390,417.102143,349.759459,373.034564,379.702323
1,2019-01-01,631.642099,551.044983,560.783324,620.092467,693.844970,549.911123,578.563016,589.688285
2,2019-01-01,169.992094,173.065674,153.552940,166.788184,181.642807,152.988182,165.419045,168.116727
3,2019-01-01,684.559821,648.783691,622.237644,675.674104,594.166542,622.932486,651.424049,668.232707
4,2019-01-01,1092.875362,965.683716,969.065554,1084.980801,1142.564632,943.483740,994.810045,1033.224703
...,...,...,...,...,...,...,...,...,...
6565,2019-12-31,917.085861,674.891174,742.331990,797.964290,830.606248,780.494827,809.243998,804.655222
6566,2019-12-31,261.569590,232.082169,210.920805,225.576610,269.498973,229.878546,235.835586,239.325998
6567,2019-12-31,1065.400121,730.457275,852.091174,925.788409,1133.560974,872.192736,899.608764,895.495983
6568,2019-12-31,1568.407214,1005.392944,1286.814471,1397.071983,2627.470943,1358.919453,1380.445879,1356.416644


In [95]:
X_test['date'] = X_test['date'].map(dt.datetime.toordinal)
# X_test = X_test.drop(columns=['huber'])

In [96]:
final_test_preds = model.predict(X_test)

In [97]:
final_tv_preds = model.predict(X)

So, with all of the model preds used, Lasso SMAPE is 4.15283882859097
- Dropping Huber only, it's 4.152469194333937
- Dropping Huber+Lasso it's 4.1525522971210345
- Dropping Lasso only, it's 4.15285802958327

So of these, definitely the best is dropping Huber only.

With all the model preds used, Ridge SMAPE is 4.158465882203197 -- that's much worse.

With all the model preds used, LinearRegression SMAPE is 4.158465902939169 -- also much worse.

Using the default hyperparams for CatBoost is even worse: 4.31463984301381

In [98]:
SMAPE(y_pred=final_tv_preds, y_true=y)

4.1397210299135585

SMAPE on all of them, with Lasso as final, Earth included, is 4.1531645410329565.

Including GBMs too gets 4.141452782674763

In [99]:
sample_df = pd.read_csv(datapath/'sample_submission.csv')

In [100]:
sample_df.loc[:, 'num_sold'] = final_test_preds

In [101]:
sample_df

,row_id,num_sold
0,26298,384.107503
1,26299,609.261468
2,26300,161.351540
3,26301,664.362471
4,26302,1066.403441
...,...,...
6565,32863,789.847219
6566,32864,219.390349
6567,32865,916.789666
6568,32866,1393.775199


In [102]:
sample_df['num_sold'] = sample_df['num_sold'].apply(round)

In [103]:
sample_df.head()

,row_id,num_sold
0,26298,384
1,26299,609
2,26300,161
3,26301,664
4,26302,1066


In [104]:
sample_df.to_csv(subpath/f"20220127_forecasts-huber-lasso+earth+gbms_lasso_preds_rounded.csv", index=False)


### Residuals

At this point, I have two DataFrames containing predictions from the forecasting models (which try to learn trends): 
1. `valid_forecast_preds`
2. `test_forecast_preds`

Both still contain the features `'date'` (having a datetime type) and the validation preds contain `num_sold`.

The goal now will be to iteratively generate a final prediction DataFrame containing all possible combinations of the forecasting model predictions and residual predictions.

In [105]:
# omitting the GBMs, since they're effectively full-stack
forecast_models = [
    'prophet', 
    'neuralprophet', 
    'ridge', 
    'linear', 
#     'huber', 
#     'lasso', 
    'earth'
] # models to use to provide basis for residual predictions

In [106]:
# encoded_all_df

In [107]:
# dump(encoded_all_df, datapath/'encoded_train+testset_with_gdp+teckmengwong-time-features+transformed-target-for-train.joblib')

In [108]:
# hybrid_train_df = encoded_all_df[:len(train_df)]
# hybrid_valid_df = encoded_all_df[len(train_df): len(train_df)+len(valid_df)]

In [109]:
# hybrid_test_df = encoded_all_df[len(train_df)+len(valid_df):]

In [110]:
# hybrid_valid_df

Note that the `tv_df` still contains both transformed targets and `num_sold`.

In [111]:
tv_df

,date,country,store,product,num_sold,gdp,month,season,wd4,wd56,...,easter50,easter51,easter52,easter53,easter54,easter55,easter56,easter57,easter58,target
0,2015-01-01,Finland,KaggleMart,Kaggle Mug,329.0,5.461456,1,1,False,False,...,False,False,False,False,False,False,False,False,False,3.738239
1,2015-01-01,Finland,KaggleMart,Kaggle Hat,520.0,5.461456,1,1,False,False,...,False,False,False,False,False,False,False,False,False,4.196010
2,2015-01-01,Finland,KaggleMart,Kaggle Sticker,146.0,5.461456,1,1,False,False,...,False,False,False,False,False,False,False,False,False,2.925788
3,2015-01-01,Finland,KaggleRama,Kaggle Mug,572.0,5.461456,1,1,False,False,...,False,False,False,False,False,False,False,False,False,4.291321
4,2015-01-01,Finland,KaggleRama,Kaggle Hat,911.0,5.461456,1,1,False,False,...,False,False,False,False,False,False,False,False,False,4.756724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26293,2018-12-31,Sweden,KaggleMart,Kaggle Hat,823.0,6.321586,12,1,False,False,...,False,False,False,False,False,False,False,False,False,4.477861
26294,2018-12-31,Sweden,KaggleMart,Kaggle Sticker,250.0,6.321586,12,1,False,False,...,False,False,False,False,False,False,False,False,False,3.286366
26295,2018-12-31,Sweden,KaggleRama,Kaggle Mug,1004.0,6.321586,12,1,False,False,...,False,False,False,False,False,False,False,False,False,4.676652
26296,2018-12-31,Sweden,KaggleRama,Kaggle Hat,1441.0,6.321586,12,1,False,False,...,False,False,False,False,False,False,False,False,False,5.037997


In [112]:
test_df

,date,country,store,product,num_sold,gdp,month,season,wd4,wd56,...,easter50,easter51,easter52,easter53,easter54,easter55,easter56,easter57,easter58,target
0,2019-01-01,Finland,KaggleMart,Kaggle Mug,NaN,5.597614,1,1,False,False,...,False,False,False,False,False,False,False,False,False,NaN
1,2019-01-01,Finland,KaggleMart,Kaggle Hat,NaN,5.597614,1,1,False,False,...,False,False,False,False,False,False,False,False,False,NaN
2,2019-01-01,Finland,KaggleMart,Kaggle Sticker,NaN,5.597614,1,1,False,False,...,False,False,False,False,False,False,False,False,False,NaN
3,2019-01-01,Finland,KaggleRama,Kaggle Mug,NaN,5.597614,1,1,False,False,...,False,False,False,False,False,False,False,False,False,NaN
4,2019-01-01,Finland,KaggleRama,Kaggle Hat,NaN,5.597614,1,1,False,False,...,False,False,False,False,False,False,False,False,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6565,2019-12-31,Sweden,KaggleMart,Kaggle Hat,NaN,6.282042,12,1,False,False,...,False,False,False,False,False,False,False,False,False,NaN
6566,2019-12-31,Sweden,KaggleMart,Kaggle Sticker,NaN,6.282042,12,1,False,False,...,False,False,False,False,False,False,False,False,False,NaN
6567,2019-12-31,Sweden,KaggleRama,Kaggle Mug,NaN,6.282042,12,1,False,False,...,False,False,False,False,False,False,False,False,False,NaN
6568,2019-12-31,Sweden,KaggleRama,Kaggle Hat,NaN,6.282042,12,1,False,False,...,False,False,False,False,False,False,False,False,False,NaN


One question: should the full `tv_df` be passed as data, or only the validation set? I think it's better to supply as much data as possible, so I'll use the whole thing for now.

In [113]:
from sklearn.model_selection import GroupKFold

In [114]:
len(tv_df)

26298

In [115]:
len(orig_train_df)

26298

In [116]:
from sklearn.metrics import mean_squared_error

In [117]:
import math

#### Residual Trainer 
(assumes Scikit-Learn API)

In [118]:
def residual_trainer(estimator, model_kwargs={}, forecast_models=forecast_models,
                     tv_df=tv_df, test_df=test_df,
                     tv_forecast_preds=tv_preds, test_forecast_preds=test_forecast_preds):
#     df_2018 = tv_df[tv_df['date'] > '2017-12-31']
    # create local versions of the dataframes, to avoid mutation
    df_train = tv_df.copy()
    df_test = test_df.copy()
    
    # apply label encoding (which Scikit-Learn models require, but *Prophets don't)
    le_dict, tv_df = label_encoder(df_train) # should leave broader scope's tv_df alone
    _, test_df = label_encoder(df_test) # should leave broader scope's test_df alone
    del df_train, df_test
    
    residual_tv_df = pd.DataFrame({
        'date': orig_train_df['date'],
        'num_sold': orig_train_df['num_sold']
    })
    print(len(residual_tv_df))
    
    residual_test_df = pd.DataFrame({
        'date': orig_test_df['date'],
    })
    
    # getting rid of unneeded 'target' feature, also num_sold since we're only interested in predicting the residual
    # following @ambrosm lightgbm nb and leaving date in for GroupKFold, but unsure of this (or if I should drop it later)
    tv_df = tv_df.drop(columns=['target', 'num_sold'])#, 'date'])
    test_df = test_df.drop(columns=['target', 'num_sold'])#, 'date'])
    
    kfolds= GroupKFold(n_splits=4)
    
    test_fold_preds = {}
    
    for forecast_model in forecast_models:
        print(f"Working with forecasts from {forecast_model}...")
        print("-----------------------------------------------------")
        forecast = tv_forecast_preds[forecast_model] # pull out the predictions on the t-v sets for a given estimator
        residuals = residual_tv_df['num_sold'] - forecast # get the residuals for the given model's forecast
#         residual_df[f'actual_{forecast_model}_residual'] = residuals # may not need to put this in there
#         tv_df['residual'] = residuals # residuals will rotate in and out of this feature
        
        
        X_test = test_df.drop(columns=['date'])
#         print("y.shape is ", y.shape)
        
        # prepare for Group K-Fold cross-val; below from @ambrosm LightGBM notebook
        oof_preds = pd.Series(0, index=tv_df.index)
#         test_preds_df = pd.DataFrame({
#             'date': test_df['date']
#         })
        score_list = []
#         params['seed'] = 1
        for fold, (train_idx, val_idx) in enumerate(kfolds.split(tv_df, groups=tv_df.date.dt.year)):
            print("-----------------------------------------------------")
            print(f"FOLD {fold}")
            X = tv_df.iloc[train_idx].drop(columns=['date'])
            y = residuals.iloc[train_idx]#['date']
            X_valid = tv_df.iloc[val_idx].drop(columns=['date'])
            y_valid = residuals.iloc[val_idx]#['date']
            
            model = estimator(**model_kwargs)
            model.fit(X, y)
            
            residual_valid_preds = model.predict(X_valid)
            residual_test_preds = model.predict(X_test)
            
            oof_preds[val_idx] = residual_valid_preds
#             test_preds_df[f'{forecast_model}_fold_{fold}_preds'] = residual_test_preds
#             residual_test_df = residual_test_df.join(test_preds_df)
#             smape = SMAPE(y_pred=residual_valid_preds, y_true=y_valid.values)
#             print(f"SMAPE: {smape}")
            rmse = math.sqrt(mean_squared_error(y_pred=residual_valid_preds, y_true=y_valid.values))
            print(f"RMSE: {rmse}")
            test_fold_preds[f'{forecast_model}_{fold}_residual_preds'] = residual_test_preds
        residual_tv_df[f'{forecast_model}_oof_residual_preds'] = oof_preds
#         residual_test_df = residual_test_df.join(test_preds_df)
    return residual_tv_df, test_fold_preds #residual_test_df
        
            
            
#             model, smape = fit_model(X_tr, X_va, run=0, fold=fold)

# print(f"Average SMAPE: {sum(score_list) / len(score_list):.5f}")
# with open('oof.pickle', 'wb') as handle: pickle.dump(oof, handle)
#         X = tv_df.drop(columns=['residual'])
#         y = tv_df['residual']
        
#         tv_
    
    
    
        
#     y_valid = valid_df['num_sold']
#     valid_df = valid_df.drop(columns=['target'])

In [119]:
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

#### Residual model hyperparams
These are inspired by the best hyperparams for the full-stack GBM models...

In [120]:
xgboost_params = {
    'tree_method': 'gpu_hist',
    'predictor': 'gpu_predictor',
    'eval_metric': ['mae', 'mape'],
    'sampling_method': 'gradient_based',
    'seed': 42,
    'grow_policy': 'lossguide',
    'learning_rate': .05,
    'max_depth': 5,
    'subsample': 0.2,
#     'max_leaves': 255,
    'lambda': 0.05,
    'alpha': 0.01,
    'n_estimators': 4200,
    'objective': 'reg:squarederror',
    'colsample_bytree': 0.98,
    'min_child_weight': 7,
    'gamma': 0.1
#     'verbose': True,
}


lightgbm_params = {
    'objective': 'mse',
    'random_state': 42,
    'device_type': 'cpu',
    'n_jobs': -1,
#                 eval_metric='auc',
#     'device_type': 'gpu',
#     'max_bin': 63, # 15 might be even better for GPU perf, but depends on dataset -- see https://lightgbm.readthedocs.io/en/latest/GPU-Performance.html
#     'gpu_use_dp': False,
    'max_depth': 3,
    'learning_rate': 0.035,
    'subsample': .96,
    'n_estimators': 6050,
    'reg_alpha': 0.009,
    'min_child_samples': 24,
    'num_leaves': 235,
    'colsample_bytree': 0.92,
}

catboost_params = {
    'task_type':'GPU',
    'silent':True,
    'random_state':42,
    'iterations': 10500,
    'learning_rate': 0.07,
    'random_strength':44,
    'od_wait': 261,
    'reg_lambda': 35.67,
    'border_count': 57,
    'min_child_samples': 19,
    'leaf_estimation_iterations': 2,
    'max_depth': 3
}
                

XGBoost SMAPEs with `n_estimators=500`:

```
26298
Working with forecasts from prophet...
-----------------------------------------------------
-----------------------------------------------------
FOLD 0
RMSE: 44.22169948290956
-----------------------------------------------------
FOLD 1
RMSE: 42.2884194618039
-----------------------------------------------------
FOLD 2
RMSE: 38.87851134388903
-----------------------------------------------------
FOLD 3
RMSE: 39.14401892347087
Working with forecasts from neuralprophet...
-----------------------------------------------------
-----------------------------------------------------
FOLD 0
RMSE: 71.59711327241503
-----------------------------------------------------
FOLD 1
RMSE: 50.410812085025206
-----------------------------------------------------
FOLD 2
RMSE: 56.30623610755547
-----------------------------------------------------
FOLD 3
RMSE: 71.87002649034811
Working with forecasts from ridge...
-----------------------------------------------------
-----------------------------------------------------
FOLD 0
RMSE: 29.951196797722172
-----------------------------------------------------
FOLD 1
RMSE: 33.87710734178301
-----------------------------------------------------
FOLD 2
RMSE: 30.3425528497887
-----------------------------------------------------
FOLD 3
RMSE: 30.908377053367243
Working with forecasts from linear...
-----------------------------------------------------
-----------------------------------------------------
FOLD 0
RMSE: 25.909733767351145
-----------------------------------------------------
FOLD 1
RMSE: 32.55539418287357
-----------------------------------------------------
FOLD 2
RMSE: 28.02573800531867
-----------------------------------------------------
FOLD 3
RMSE: 27.512157750399876
Working with forecasts from huber...
-----------------------------------------------------
-----------------------------------------------------
FOLD 0
RMSE: 43.8980188030959
-----------------------------------------------------
FOLD 1
RMSE: 55.40375581530791
-----------------------------------------------------
FOLD 2
RMSE: 46.37734856211295
-----------------------------------------------------
FOLD 3
RMSE: 41.743947334147386
Working with forecasts from lasso...
-----------------------------------------------------
-----------------------------------------------------
FOLD 0
RMSE: 42.212939444847706
-----------------------------------------------------
FOLD 1
RMSE: 58.6828784233263
-----------------------------------------------------
FOLD 2
RMSE: 47.91103516420516
-----------------------------------------------------
FOLD 3
RMSE: 41.89282012115221
```

#### Residual model training / loading

In [121]:
%%time 
# xgb_tv_df, xgb_test_fold_preds = residual_trainer(estimator=XGBRegressor, model_kwargs=xgboost_params,)
# dump(xgb_tv_df, predpath/'20220127_-huber-lasso+earth_residual_xgboost_oof_preds.joblib')
# dump(xgb_test_fold_preds, predpath/'20220125_-huber-lasso+earth_residual_xgboost_test_preds.joblib')
xgb_tv_df = load(predpath/'20220127_-huber-lasso+earth_residual_xgboost_oof_preds.joblib')
xgb_test_fold_preds = load(predpath/'20220125_-huber-lasso+earth_residual_xgboost_test_preds.joblib')
# loading the 500-estimator version, which is universally better
# xgb_tv_df = load(predpath/'20220124_residual_oof_xgboost_preds.joblib')
# xgb_test_fold_preds = load(predpath/'20220124_residual_test_xgboost_preds.joblib')

CPU times: user 1.23 ms, sys: 4.02 ms, total: 5.25 ms
Wall time: 5.23 ms


In [122]:
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

In [123]:
%%time
# lgb_tv_df, lgb_test_fold_preds = residual_trainer(estimator=LGBMRegressor, model_kwargs=lightgbm_params,)
# dump(lgb_tv_df, predpath/'20220127_-huber-lasso+earth_residual_lightgbm_oof_preds.joblib')
# dump(lgb_test_fold_preds, predpath/'20220127_-huber-lasso+earth_residual_lightgbm_test_preds.joblib')

lgb_tv_df = load(predpath/'20220127_-huber-lasso+earth_residual_lightgbm_oof_preds.joblib')
lgb_test_fold_preds = load(predpath/'20220127_-huber-lasso+earth_residual_lightgbm_test_preds.joblib')
# lgb_tv_df = load(predpath/'20220124_residual_oof_residual_lightgbm_preds.joblib')
# lgb_test_fold_preds = load(predpath/'20220124_residual_test_residual_lightgbm_preds.joblib')

CPU times: user 4.56 ms, sys: 0 ns, total: 4.56 ms
Wall time: 4.46 ms


In [124]:
%%time
# cat_tv_df, cat_test_fold_preds = residual_trainer(estimator=CatBoostRegressor, model_kwargs=catboost_params,)
# dump(cat_tv_df, predpath/'20220127_-huber-lasso+earth_residual_catboost_oof_preds.joblib')
# dump(cat_test_df, predpath/'20220127_-huber-lasso+earth_residual_catboost_test_preds.joblib')

cat_tv_df = load(predpath/'20220127_-huber-lasso+earth_residual_catboost_oof_preds.joblib')
cat_test_fold_preds = load(predpath/'20220127_-huber-lasso+earth_residual_catboost_test_preds.joblib')

# dump(cat_tv_df, predpath/'20220125_+earth_residual_oof_residual_catboost_preds.joblib')
# dump(cat_test_fold_preds, predpath/'20220125_+earth_residual_test_residual_catboost_preds.joblib')
# cat_tv_df = load(predpath/'20220124_residual_oof_residual_catboost_preds.joblib')
# cat_test_fold_preds = load(predpath/'20220124_residual_test_residual_catboost_preds.joblib')

CPU times: user 4.93 ms, sys: 7 µs, total: 4.94 ms
Wall time: 4.64 ms


In [125]:
# dump(cat_test_fold_preds, predpath/'20220127_-huber-lasso+earth_residual_catboost_test_preds.joblib')


In [126]:
# %time
# ridge_tv_df, ridge_test_fold_preds = residual_trainer(estimator=Ridge)

In [127]:
# %time
# lasso_tv_df, lasso_test_fold_preds = residual_trainer(estimator=Lasso)

In [128]:
# %time
# linear_tv_df, linear_test_fold_preds = residual_trainer(estimator=LinearRegression)

In [129]:
# dump(valid_forecast_preds, predpath/'20220124_forecast_valid2018_preds.joblib')
# dump(test_forecast_preds, predpath/'20220124_forecast_test_preds.joblib')

#### Assembling together residual predictions

In [130]:
tv_dfs = {
    'xgb': xgb_tv_df, 
    'lgb': lgb_tv_df, 
    'cat': cat_tv_df,
#     'ridge': ridge_tv_df,
#     'lasso': lasso_tv_df,
#     'linear': linear_tv_df,
}

for arch in tv_dfs.keys():
    for forecast_model in forecast_models:
        tv_dfs[arch][f'{forecast_model}_pred'] = tv_preds[forecast_model]
        tv_dfs[arch][f'{forecast_model}_residual'] = tv_dfs[arch]['num_sold'] - tv_preds[forecast_model] #- xgb_tv_df['num_sold']

In [131]:
# xgb_tv_df

In [132]:
tv_dfs['xgb']

,date,num_sold,prophet_oof_residual_preds,neuralprophet_oof_residual_preds,ridge_oof_residual_preds,linear_oof_residual_preds,earth_oof_residual_preds,prophet_pred,prophet_residual,neuralprophet_pred,neuralprophet_residual,ridge_pred,ridge_residual,linear_pred,linear_residual,earth_pred,earth_residual
0,2015-01-01,329,-11.155267,-3.433359,41.070728,2.167291,38.581459,346.560416,-17.560416,329.134521,-0.134521,283.272258,45.727742,324.188332,4.811668,360.672190,-31.672190
1,2015-01-01,520,-28.441416,5.382236,61.150066,-2.865466,13.708234,536.203586,-16.203586,458.008301,61.991699,439.771734,80.228266,503.081569,16.918431,584.851806,-64.851806
2,2015-01-01,146,-6.331515,2.279078,17.805452,3.196031,12.222075,143.412803,2.587197,145.325577,0.674423,120.885683,25.114317,136.667932,9.332068,138.401176,7.598824
3,2015-01-01,572,6.812320,-0.307026,69.837616,-4.111918,61.455334,590.117165,-18.117165,552.571167,19.428833,488.810710,83.189290,555.531957,16.468043,562.479785,9.520215
4,2015-01-01,911,0.980131,81.692017,118.799637,-7.382885,17.165312,939.673009,-28.673009,781.967285,129.032715,771.701702,139.298298,890.469201,20.530799,944.584508,-33.584508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26293,2018-12-31,823,-55.787830,259.895264,140.673294,-8.261147,-16.538561,898.322121,-75.322121,669.418457,153.581543,723.714830,99.285170,846.597603,-23.597603,836.947940,-13.947940
26294,2018-12-31,250,2.033622,24.113207,50.021725,9.767989,1.243830,253.512355,-3.512355,227.158142,22.841858,205.880079,44.119921,241.048951,8.951049,254.287204,-4.287204
26295,2018-12-31,1004,-47.085091,349.409943,150.556046,-3.797827,-20.398714,1039.635205,-35.635205,715.639648,288.360352,832.192362,171.807638,975.785339,28.214661,1058.174059,-54.174059
26296,2018-12-31,1441,-21.645599,593.559570,221.251251,-4.062232,-15.556402,1526.908216,-85.908216,980.234009,460.765991,1255.885410,185.114590,1468.776593,-27.776593,2036.549357,-595.549357


In [133]:
for arch in tv_dfs.keys(): # xgb, then lgb, then cat
    print(f"\nFor {arch}...\n")
    for forecast_model in forecast_models:
        
#         if arch == 'xgb':
#             adjusted_smape = SMAPE(y_pred=tv_dfs[arch][f'{forecast_model}_pred']+tv_dfs[arch][f'{forecast_model}_oof_preds'], y_true=tv_dfs[arch]['num_sold'])
#         else:
        adjusted_smape = SMAPE(y_pred=tv_dfs[arch][f'{forecast_model}_pred']+tv_dfs[arch][f'{forecast_model}_oof_residual_preds'], y_true=tv_dfs[arch]['num_sold'])
        original_smape = SMAPE(y_pred=tv_dfs[arch][f'{forecast_model}_pred'], y_true=tv_dfs[arch]['num_sold'])
        print(f'Before residual adjustment, SMAPE for {forecast_model} was {original_smape}. Final SMAPE for {forecast_model} is {adjusted_smape}.')


For xgb...

Before residual adjustment, SMAPE for prophet was 7.098824805150043. Final SMAPE for prophet is 6.026884283769224.
Before residual adjustment, SMAPE for neuralprophet was 7.926662675729486. Final SMAPE for neuralprophet is 8.772834447498484.
Before residual adjustment, SMAPE for ridge was 4.781041939812455. Final SMAPE for ridge is 5.289615055217569.
Before residual adjustment, SMAPE for linear was 4.175573461450916. Final SMAPE for linear is 5.040366554565315.
Before residual adjustment, SMAPE for earth was 5.338351055481194. Final SMAPE for earth is 6.029509457078209.

For lgb...

Before residual adjustment, SMAPE for prophet was 7.098824805150043. Final SMAPE for prophet is 6.122509629258212.
Before residual adjustment, SMAPE for neuralprophet was 7.926662675729486. Final SMAPE for neuralprophet is 8.460500050162052.
Before residual adjustment, SMAPE for ridge was 4.781041939812455. Final SMAPE for ridge is 4.876170971167097.
Before residual adjustment, SMAPE for linear

In [134]:
tv_preds

,date,num_sold,prophet,neuralprophet,ridge,linear,earth,xgboost,lightgbm,catboost
0,2015-01-01,329,346.560416,329.134521,283.272258,324.188332,360.672190,343.250478,331.496226,326.054356
1,2015-01-01,520,536.203586,458.008301,439.771734,503.081569,584.851806,510.616623,514.576867,509.005936
2,2015-01-01,146,143.412803,145.325577,120.885683,136.667932,138.401176,148.959080,146.429925,144.894609
3,2015-01-01,572,590.117165,552.571167,488.810710,555.531957,562.479785,596.463984,584.653899,569.038022
4,2015-01-01,911,939.673009,781.967285,771.701702,890.469201,944.584508,907.745110,888.842963,882.892654
...,...,...,...,...,...,...,...,...,...,...
26293,2018-12-31,823,898.322121,669.418457,723.714830,846.597603,836.947940,830.633157,814.597001,825.377960
26294,2018-12-31,250,253.512355,227.158142,205.880079,241.048951,254.287204,241.115195,234.409908,245.208674
26295,2018-12-31,1004,1039.635205,715.639648,832.192362,975.785339,1058.174059,898.057926,900.745370,915.158956
26296,2018-12-31,1441,1526.908216,980.234009,1255.885410,1468.776593,2036.549357,1397.507711,1419.978895,1401.414363


## Submission of Adjusted XGB Naivish

In [135]:
# oof_features = ['']
adjusted_oof_preds = pd.DataFrame({
    'date': orig_train_df['date'],
})

tv_dfs_dict = {
    'xgb': xgb_tv_df,
    'lgb': lgb_tv_df,
    'cat': cat_tv_df,
}

for arch in tv_dfs_dict.keys():
    print(f"{arch}\n")
    for forecast_model in forecast_models:
#         if arch == 'xgb':
#                         adjusted_oof_preds[f"{arch}+{forecast_model}"] = tv_dfs_dict[arch][f'{forecast_model}_pred']+tv_dfs_dict[arch][f'{forecast_model}_oof_preds']
#         else:
        adjusted_oof_preds[f"{arch}+{forecast_model}"] = tv_dfs_dict[arch][f'{forecast_model}_pred']+tv_dfs_dict[arch][f'{forecast_model}_oof_residual_preds']

xgb

lgb

cat



In [136]:
tv_preds_only = tv_preds.drop(columns=['date'])

In [137]:
adjusted_oof_preds_selective = adjusted_oof_preds.drop(columns=[
                                                                'xgb+neuralprophet', 
#                                                                 'xgb+ridge', 
#                                                                 'xgb+linear', 
                                                                'xgb+earth',
                                                                'lgb+neuralprophet', 
#                                                                 'lgb+ridge', 
#                                                                 'lgb+linear', 
                                                                'lgb+earth',
                                                                'cat+neuralprophet', 
#                                                                 'cat+linear', 
                                                                'cat+earth'
                                                               ]   
                                                      )
                                                      
# tv_preds_only = tv_preds.drop(columns=['date', 'num_sold'])
adjusted_oof_preds_expanded = adjusted_oof_preds.join(tv_preds_only)#.drop(columns=['date', 'num_sold'], inplace=True))

In [138]:
adjusted_oof_preds_selective_expanded = adjusted_oof_preds_selective.join(tv_preds_only)
adjusted_oof_preds_selective_expanded

,date,xgb+prophet,xgb+ridge,xgb+linear,lgb+prophet,lgb+ridge,lgb+linear,cat+prophet,cat+ridge,cat+linear,num_sold,prophet,neuralprophet,ridge,linear,earth,xgboost,lightgbm,catboost
0,2015-01-01,335.405149,324.342987,326.355622,353.816288,315.109073,320.781832,342.162795,308.871192,319.407647,329,346.560416,329.134521,283.272258,324.188332,360.672190,343.250478,331.496226,326.054356
1,2015-01-01,507.762170,500.921801,500.216103,502.099511,502.309507,497.514037,505.468074,494.362517,498.780657,520,536.203586,458.008301,439.771734,503.081569,584.851806,510.616623,514.576867,509.005936
2,2015-01-01,137.081288,138.691135,139.863963,161.207516,137.984103,133.714742,149.287358,134.114025,135.374514,146,143.412803,145.325577,120.885683,136.667932,138.401176,148.959080,146.429925,144.894609
3,2015-01-01,596.929485,558.648326,551.420039,577.112455,555.405898,552.703990,576.479862,555.407912,550.887544,572,590.117165,552.571167,488.810710,555.531957,562.479785,596.463984,584.653899,569.038022
4,2015-01-01,940.653140,890.501339,883.086316,934.119140,893.043397,882.242292,919.623890,886.321175,881.045421,911,939.673009,781.967285,771.701702,890.469201,944.584508,907.745110,888.842963,882.892654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26293,2018-12-31,842.534291,864.388124,838.336456,924.978028,855.545431,839.722064,905.346443,841.013156,845.194035,823,898.322121,669.418457,723.714830,846.597603,836.947940,830.633157,814.597001,825.377960
26294,2018-12-31,255.545977,255.901804,250.816940,189.916929,261.353502,241.161049,206.318911,244.486973,242.723551,250,253.512355,227.158142,205.880079,241.048951,254.287204,241.115195,234.409908,245.208674
26295,2018-12-31,992.550115,982.748408,971.987512,1006.865069,980.987703,965.731059,1031.550009,956.716929,972.733624,1004,1039.635205,715.639648,832.192362,975.785339,1058.174059,898.057926,900.745370,915.158956
26296,2018-12-31,1505.262616,1477.136661,1464.714361,1644.013144,1473.934875,1460.599837,1614.987267,1448.806267,1466.727788,1441,1526.908216,980.234009,1255.885410,1468.776593,2036.549357,1397.507711,1419.978895,1401.414363


In [139]:
adjusted_oof_preds_expanded

,date,xgb+prophet,xgb+neuralprophet,xgb+ridge,xgb+linear,xgb+earth,lgb+prophet,lgb+neuralprophet,lgb+ridge,lgb+linear,...,cat+earth,num_sold,prophet,neuralprophet,ridge,linear,earth,xgboost,lightgbm,catboost
0,2015-01-01,335.405149,325.701162,324.342987,326.355622,399.253649,353.816288,303.237210,315.109073,320.781832,...,378.548520,329,346.560416,329.134521,283.272258,324.188332,360.672190,343.250478,331.496226,326.054356
1,2015-01-01,507.762170,463.390537,500.921801,500.216103,598.560040,502.099511,423.579452,502.309507,497.514037,...,592.709651,520,536.203586,458.008301,439.771734,503.081569,584.851806,510.616623,514.576867,509.005936
2,2015-01-01,137.081288,147.604655,138.691135,139.863963,150.623251,161.207516,152.517721,137.984103,133.714742,...,122.792138,146,143.412803,145.325577,120.885683,136.667932,138.401176,148.959080,146.429925,144.894609
3,2015-01-01,596.929485,552.264141,558.648326,551.420039,623.935119,577.112455,547.578908,555.405898,552.703990,...,610.328935,572,590.117165,552.571167,488.810710,555.531957,562.479785,596.463984,584.653899,569.038022
4,2015-01-01,940.653140,863.659302,890.501339,883.086316,961.749819,934.119140,918.265905,893.043397,882.242292,...,962.248311,911,939.673009,781.967285,771.701702,890.469201,944.584508,907.745110,888.842963,882.892654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26293,2018-12-31,842.534291,929.313721,864.388124,838.336456,820.409379,924.978028,968.214591,855.545431,839.722064,...,834.502360,823,898.322121,669.418457,723.714830,846.597603,836.947940,830.633157,814.597001,825.377960
26294,2018-12-31,255.545977,251.271349,255.901804,250.816940,255.531034,189.916929,244.527543,261.353502,241.161049,...,256.669182,250,253.512355,227.158142,205.880079,241.048951,254.287204,241.115195,234.409908,245.208674
26295,2018-12-31,992.550115,1065.049591,982.748408,971.987512,1037.775345,1006.865069,1031.111582,980.987703,965.731059,...,1014.550436,1004,1039.635205,715.639648,832.192362,975.785339,1058.174059,898.057926,900.745370,915.158956
26296,2018-12-31,1505.262616,1573.793579,1477.136661,1464.714361,2020.992955,1644.013144,1589.240447,1473.934875,1460.599837,...,2042.430342,1441,1526.908216,980.234009,1255.885410,1468.776593,2036.549357,1397.507711,1419.978895,1401.414363


In [140]:
xgb_test_preds_df = pd.DataFrame(xgb_test_fold_preds)
lgb_test_preds_df = pd.DataFrame(lgb_test_fold_preds)
cat_test_preds_df = pd.DataFrame(cat_test_fold_preds)

test_dfs_dict = {
    'xgb': xgb_test_preds_df,
    'lgb': lgb_test_preds_df,
    'cat': cat_test_preds_df,
}

In [141]:
xgb_test_preds_df

,prophet_0_residual_preds,prophet_1_residual_preds,prophet_2_residual_preds,prophet_3_residual_preds,neuralprophet_0_residual_preds,neuralprophet_1_residual_preds,neuralprophet_2_residual_preds,neuralprophet_3_residual_preds,ridge_0_residual_preds,ridge_1_residual_preds,ridge_2_residual_preds,ridge_3_residual_preds,linear_0_residual_preds,linear_1_residual_preds,linear_2_residual_preds,linear_3_residual_preds,earth_0_residual_preds,earth_1_residual_preds,earth_2_residual_preds,earth_3_residual_preds
0,-17.033663,-21.819101,-20.461470,4.619490,-21.076921,3.158182,0.283048,11.436007,41.045731,43.582924,39.203148,52.930038,11.043393,-10.374799,-3.198138,11.198472,6.389496,12.277696,9.860520,44.748325
1,-7.953610,-32.106422,-29.866251,-13.119456,23.624197,17.082817,12.703603,25.939672,79.355980,60.213955,57.458580,68.211830,13.730322,-5.473454,-6.398356,-0.480903,15.075122,-1.671091,-11.525816,26.900377
2,-3.117981,-17.535273,-14.463274,-1.783982,-6.031038,9.711327,-7.441924,5.877184,7.184831,9.987694,8.546426,18.454214,1.068279,-9.214305,-4.012601,6.126245,-0.055850,3.943094,5.865566,11.813078
3,-17.838993,-29.329393,-4.876259,19.027905,-15.885752,-12.469327,3.310822,14.986485,75.993629,66.369682,75.367096,75.453247,4.767874,-20.277174,13.027990,1.270042,58.526283,44.996883,42.497440,72.461845
4,-26.509970,-16.158661,-10.763204,10.099065,65.650642,121.963394,108.657166,100.493484,117.317764,127.078529,131.368958,118.615768,-13.505404,-6.629692,11.166040,-11.286895,8.115604,12.812317,7.845061,29.399580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6565,-99.123657,-71.305107,-59.250942,-10.330985,138.355927,196.987167,150.624374,262.165802,101.768959,119.761124,108.255318,130.089218,-19.740946,-3.063937,-4.934291,15.305479,6.367306,-12.736743,-19.809973,-51.501400
6566,-16.493229,-16.453074,-14.665970,13.453952,-38.068619,8.731948,4.070496,54.360695,42.092594,35.633297,36.126293,46.406498,15.607967,12.405040,10.690983,13.768815,7.738034,5.247971,19.875986,-1.179993
6567,-87.356529,-57.674362,-53.783466,20.081490,210.766830,250.296646,213.821228,337.864075,139.503235,132.930740,140.493744,161.274094,6.022972,-0.797151,1.846847,38.112049,7.979291,3.600235,-11.092898,-10.844264
6568,-75.296303,-29.718555,-70.626648,28.712942,445.830048,494.277771,396.313354,517.456787,199.091965,202.822571,184.382248,201.019592,-11.430958,0.492131,-22.447880,-5.213744,-32.852684,-4.504761,-79.522964,-133.354050


In [142]:
lgb_test_preds_df

,prophet_0_residual_preds,prophet_1_residual_preds,prophet_2_residual_preds,prophet_3_residual_preds,neuralprophet_0_residual_preds,neuralprophet_1_residual_preds,neuralprophet_2_residual_preds,neuralprophet_3_residual_preds,ridge_0_residual_preds,ridge_1_residual_preds,ridge_2_residual_preds,ridge_3_residual_preds,linear_0_residual_preds,linear_1_residual_preds,linear_2_residual_preds,linear_3_residual_preds,earth_0_residual_preds,earth_1_residual_preds,earth_2_residual_preds,earth_3_residual_preds
0,8.417380,-61.508158,23.100475,34.776261,14.015606,-67.475469,24.595720,17.191331,42.890087,29.066765,51.724626,50.557733,-25.086643,-6.080084,-21.387555,-18.124663,1.457896,12.600928,5.106946,27.319493
1,-0.405302,-49.418148,-10.470536,-8.375791,68.692888,-1.427907,73.688546,27.369757,79.817463,63.870581,76.631124,75.445836,-13.752755,1.513884,-18.366406,-21.613810,-1.088331,2.642132,-12.250419,4.065161
2,17.998711,-25.421969,30.043784,39.225742,14.824780,-30.362008,23.805578,24.665745,26.499811,11.961173,23.278174,24.414215,-10.134384,-3.428975,-14.746318,-10.119366,2.341616,14.972660,-1.494605,11.773964
3,-6.633334,-62.285334,18.500085,11.427457,44.873675,-14.800776,69.548091,47.822503,78.918727,67.520122,101.831534,85.488041,-37.633571,-6.703128,-17.667744,-29.790984,34.890573,15.067120,40.353713,54.879569
4,16.107800,-11.843453,15.149475,17.086193,200.071190,175.228765,229.800475,207.703967,130.247401,131.723874,152.688608,133.778653,-39.749207,-2.612528,-18.533138,-36.136083,0.623418,0.111770,4.338212,14.705306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6565,-34.769136,26.655907,-30.118321,31.958759,224.130400,298.796134,157.606998,286.654485,121.092300,131.830601,111.756038,139.203643,-0.500225,-6.875539,-6.603869,5.567733,4.104113,-3.600418,-17.342312,-21.125879
6566,-35.256477,-63.595426,-56.630480,-13.999866,-4.979816,17.369401,-28.662670,20.092102,50.968948,55.473423,46.283016,51.097698,7.056347,0.112098,6.443867,8.402282,13.542317,13.475763,4.569655,8.916277
6567,-28.233504,-32.770136,-30.336255,41.386606,298.183710,315.471933,224.615620,357.176083,160.322462,148.795341,144.805298,175.504044,18.861284,-10.054280,12.646866,18.732640,-14.994401,-14.018352,-0.518033,-25.635188
6568,30.104592,117.104928,46.801813,156.275600,521.559478,609.006438,414.563545,634.022354,197.029132,218.049465,190.814035,225.541757,-8.550275,-8.176756,-11.061050,-6.942820,-56.755817,-12.915289,-54.205188,-87.964328


In [143]:
adjusted_oof_preds['num_sold'] = orig_train_df['num_sold']

In [144]:
gbms_and_adjusted_oof_preds = adjusted_oof_preds.copy()
gbms_and_adjusted_oof_preds['xgb'] = xgb_forecast_oof_preds
gbms_and_adjusted_oof_preds['lgb'] = lgb_forecast_oof_preds
gbms_and_adjusted_oof_preds['cat'] = cat_forecast_oof_preds

In [145]:
gbms_and_adjusted_oof_preds

,date,xgb+prophet,xgb+neuralprophet,xgb+ridge,xgb+linear,xgb+earth,lgb+prophet,lgb+neuralprophet,lgb+ridge,lgb+linear,lgb+earth,cat+prophet,cat+neuralprophet,cat+ridge,cat+linear,cat+earth,num_sold,xgb,lgb,cat
0,2015-01-01,335.405149,325.701162,324.342987,326.355622,399.253649,353.816288,303.237210,315.109073,320.781832,360.017842,342.162795,297.299018,308.871192,319.407647,378.548520,329,343.250478,331.496226,326.054356
1,2015-01-01,507.762170,463.390537,500.921801,500.216103,598.560040,502.099511,423.579452,502.309507,497.514037,575.971519,505.468074,453.105968,494.362517,498.780657,592.709651,520,510.616623,514.576867,509.005936
2,2015-01-01,137.081288,147.604655,138.691135,139.863963,150.623251,161.207516,152.517721,137.984103,133.714742,126.248183,149.287358,153.054598,134.114025,135.374514,122.792138,146,148.959080,146.429925,144.894609
3,2015-01-01,596.929485,552.264141,558.648326,551.420039,623.935119,577.112455,547.578908,555.405898,552.703990,582.508689,576.479862,554.053877,555.407912,550.887544,610.328935,572,596.463984,584.653899,569.038022
4,2015-01-01,940.653140,863.659302,890.501339,883.086316,961.749819,934.119140,918.265905,893.043397,882.242292,937.060549,919.623890,887.265360,886.321175,881.045421,962.248311,911,907.745110,888.842963,882.892654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26293,2018-12-31,842.534291,929.313721,864.388124,838.336456,820.409379,924.978028,968.214591,855.545431,839.722064,833.347522,905.346443,910.465738,841.013156,845.194035,834.502360,823,830.633157,814.597001,825.377960
26294,2018-12-31,255.545977,251.271349,255.901804,250.816940,255.531034,189.916929,244.527543,261.353502,241.161049,267.762967,206.318911,223.127072,244.486973,242.723551,256.669182,250,241.115195,234.409908,245.208674
26295,2018-12-31,992.550115,1065.049591,982.748408,971.987512,1037.775345,1006.865069,1031.111582,980.987703,965.731059,1044.155707,1031.550009,986.525329,956.716929,972.733624,1014.550436,1004,898.057926,900.745370,915.158956
26296,2018-12-31,1505.262616,1573.793579,1477.136661,1464.714361,2020.992955,1644.013144,1589.240447,1473.934875,1460.599837,2023.634069,1614.987267,1489.943014,1448.806267,1466.727788,2042.430342,1441,1397.507711,1419.978895,1401.414363


In [146]:
model = Lasso()

In [147]:
X = adjusted_oof_preds.drop(columns=['num_sold'])
y = adjusted_oof_preds['num_sold']

In [148]:
adjusted_residual_test_preds = pd.DataFrame({
    'date': orig_test_df['date']
})

# tv_dfs_dict = {
#     'xgb': xgb_tv_df,
#     'lgb': lgb_tv_df,
#     'cat': cat_tv_df,
# }

for arch in test_dfs_dict.keys():#['xgb', 'lgb', 'cat']: #prophet_0_residual_preds
    for forecast_model in forecast_models:
        adjusted_residual_test_preds[f'{arch}+{forecast_model}'] = sum([test_dfs_dict[arch][f'{forecast_model}_{i}_residual_preds'] for i in range(4)]) / 4
adjusted_residual_test_preds

,date,xgb+prophet,xgb+neuralprophet,xgb+ridge,xgb+linear,xgb+earth,lgb+prophet,lgb+neuralprophet,lgb+ridge,lgb+linear,lgb+earth,cat+prophet,cat+neuralprophet,cat+ridge,cat+linear,cat+earth
0,2019-01-01,-13.673685,-1.549922,44.190460,2.167232,18.319008,1.196489,-2.918203,43.559803,-17.669736,11.621316,0.955770,-4.563203,40.854708,-17.643456,14.989288
1,2019-01-01,-20.761435,19.837572,66.310081,0.344402,7.194648,-17.167444,42.080821,73.941251,-13.054772,-1.657864,-19.494950,31.390587,73.837511,-14.630494,-6.618691
2,2019-01-01,-9.225127,0.528887,11.043291,-1.508096,5.391472,15.461567,8.233524,21.538343,-9.607261,6.898409,17.580708,21.197725,20.440282,-9.436740,-2.127463
3,2019-01-01,-8.254186,-2.514443,73.295914,-0.302817,54.620613,-9.747781,36.860873,83.439606,-22.948857,36.297744,-10.562935,21.469288,80.422080,-22.815989,44.181862
4,2019-01-01,-10.833193,99.191177,123.595261,-5.063988,14.543140,9.125004,203.201099,137.109634,-24.257739,4.944676,-7.329994,152.278801,131.605603,-22.844062,5.667452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6565,2019-12-31,-60.002674,187.033325,114.968658,-3.108424,-19.420202,-1.568198,241.797005,125.970645,-2.102975,-9.491124,-33.207248,221.947118,115.450504,-2.479457,-7.796854
6566,2019-12-31,-8.539580,7.273630,40.064671,13.118201,7.920500,-42.370562,0.954754,50.955771,5.503649,10.126003,-56.678126,-6.542535,41.053457,4.860815,10.829827
6567,2019-12-31,-44.683212,253.187195,143.550446,11.296180,-2.589409,-12.488322,298.861837,157.356786,10.046627,-13.791494,-21.789688,260.933804,137.885133,7.177214,-31.752921
6568,2019-12-31,-36.732143,463.469482,196.829102,-9.650113,-62.558617,87.571733,544.787954,207.858597,-8.682725,-52.960155,36.306836,475.671818,189.814122,-6.990601,-46.545445


In [149]:
test_forecast_preds

,date,prophet,neuralprophet,ridge,linear,earth,xgboost,lightgbm,catboost
0,737060,402.296576,383.543457,359.817081,392.328390,417.102143,349.759459,373.034564,379.702323
1,737060,631.642099,551.044983,560.783324,620.092467,693.844970,549.911123,578.563016,589.688285
2,737060,169.992094,173.065674,153.552940,166.788184,181.642807,152.988182,165.419045,168.116727
3,737060,684.559821,648.783691,622.237644,675.674104,594.166542,622.932486,651.424049,668.232707
4,737060,1092.875362,965.683716,969.065554,1084.980801,1142.564632,943.483740,994.810045,1033.224703
...,...,...,...,...,...,...,...,...,...
6565,737424,917.085861,674.891174,742.331990,797.964290,830.606248,780.494827,809.243998,804.655222
6566,737424,261.569590,232.082169,210.920805,225.576610,269.498973,229.878546,235.835586,239.325998
6567,737424,1065.400121,730.457275,852.091174,925.788409,1133.560974,872.192736,899.608764,895.495983
6568,737424,1568.407214,1005.392944,1286.814471,1397.071983,2627.470943,1358.919453,1380.445879,1356.416644


In [150]:
dump(adjusted_residual_test_preds, predpath/'20220127_residual_test_predictions+earth_averaged_by_fold_using_GBMs.joblib')

['/media/sf/easystore/kaggle_data/tabular_playgrounds/jan2022/preds/20220127_residual_test_predictions+earth_averaged_by_fold_using_GBMs.joblib']

In [151]:
dump(adjusted_oof_preds, predpath/'20220127_final_oof_predictions_forecast+Earth+GBMs-residuals.joblib')

['/media/sf/easystore/kaggle_data/tabular_playgrounds/jan2022/preds/20220127_final_oof_predictions_forecast+Earth+GBMs-residuals.joblib']

In [152]:
# adjusted_test_residual_preds = adjusted_test_preds.copy() # fixing the confusing name

In [153]:
adjusted_test_preds = pd.DataFrame({
    'date': orig_test_df['date']
})

for arch in ['xgb', 'lgb', 'cat']: # ['xgb', 'lgb', 'cat']:
    for forecast_model in forecast_models:
        adjusted_test_preds[f'{arch}+{forecast_model}'] = test_forecast_preds[f'{forecast_model}'] + adjusted_residual_test_preds[f'{arch}+{forecast_model}']

In [154]:
adjusted_test_preds

,date,xgb+prophet,xgb+neuralprophet,xgb+ridge,xgb+linear,xgb+earth,lgb+prophet,lgb+neuralprophet,lgb+ridge,lgb+linear,lgb+earth,cat+prophet,cat+neuralprophet,cat+ridge,cat+linear,cat+earth
0,2019-01-01,388.622891,381.993536,404.007541,394.495622,435.421150,403.493065,380.625254,403.376884,374.658654,428.723458,403.252346,378.980254,400.671789,374.684934,432.091431
1,2019-01-01,610.880664,570.882555,627.093405,620.436869,701.039618,614.474655,593.125804,634.724575,607.037695,692.187106,612.147149,582.435570,634.620835,605.461973,687.226279
2,2019-01-01,160.766967,173.594561,164.596231,165.280088,187.034279,185.453661,181.299197,175.091284,157.180923,188.541216,187.572802,194.263398,173.993223,157.351444,179.515344
3,2019-01-01,676.305636,646.269248,695.533558,675.371287,648.787155,674.812040,685.644565,705.677250,652.725247,630.464286,673.996886,670.252980,702.659724,652.858116,638.348403
4,2019-01-01,1082.042169,1064.874893,1092.660815,1079.916814,1157.107772,1102.000366,1168.884815,1106.175188,1060.723062,1147.509309,1085.545368,1117.962517,1100.671157,1062.136740,1148.232084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6565,2019-12-31,857.083187,861.924500,857.300648,794.855865,811.186046,915.517664,916.688179,868.302635,795.861315,821.115124,883.878613,896.838292,857.782493,795.484833,822.809394
6566,2019-12-31,253.030010,239.355799,250.985476,238.694811,277.419473,219.199028,233.036923,261.876577,231.080258,279.624976,204.891464,225.539633,251.974263,230.437425,280.328800
6567,2019-12-31,1020.716908,983.644470,995.641620,937.084589,1130.971565,1052.911798,1029.319112,1009.447960,935.835037,1119.769480,1043.610433,991.391079,989.976308,932.965624,1101.808053
6568,2019-12-31,1531.675071,1468.862427,1483.643573,1387.421869,2564.912326,1655.978947,1550.180898,1494.673068,1388.389257,2574.510788,1604.714050,1481.064763,1476.628593,1390.081381,2580.925498


In [155]:
test_forecast_preds_only = test_forecast_preds.drop(columns=['date'])

In [156]:
adjusted_test_preds_selective = adjusted_test_preds.drop(columns=[
                                                                'xgb+neuralprophet', 
#                                                                 'xgb+ridge', 
#                                                                 'xgb+linear', 
                                                                'xgb+earth',
                                                                'lgb+neuralprophet', 
#                                                                 'lgb+ridge', 
#                                                                 'lgb+linear', 
                                                                'lgb+earth',
                                                                'cat+neuralprophet', 
#                                                                 'cat+linear', 
                                                                'cat+earth'
                                                               ]   
                                                      )
                                                      
# tv_preds_only = tv_preds.drop(columns=['date', 'num_sold'])
adjusted_test_preds_expanded = adjusted_test_preds.join(test_forecast_preds_only)#_only)#.drop(columns=['date', 'num_sold'], inplace=True))

In [157]:
adjusted_test_preds_selective_expanded = adjusted_test_preds_selective.join(test_forecast_preds_only)
adjusted_test_preds_selective_expanded

,date,xgb+prophet,xgb+ridge,xgb+linear,lgb+prophet,lgb+ridge,lgb+linear,cat+prophet,cat+ridge,cat+linear,prophet,neuralprophet,ridge,linear,earth,xgboost,lightgbm,catboost
0,2019-01-01,388.622891,404.007541,394.495622,403.493065,403.376884,374.658654,403.252346,400.671789,374.684934,402.296576,383.543457,359.817081,392.328390,417.102143,349.759459,373.034564,379.702323
1,2019-01-01,610.880664,627.093405,620.436869,614.474655,634.724575,607.037695,612.147149,634.620835,605.461973,631.642099,551.044983,560.783324,620.092467,693.844970,549.911123,578.563016,589.688285
2,2019-01-01,160.766967,164.596231,165.280088,185.453661,175.091284,157.180923,187.572802,173.993223,157.351444,169.992094,173.065674,153.552940,166.788184,181.642807,152.988182,165.419045,168.116727
3,2019-01-01,676.305636,695.533558,675.371287,674.812040,705.677250,652.725247,673.996886,702.659724,652.858116,684.559821,648.783691,622.237644,675.674104,594.166542,622.932486,651.424049,668.232707
4,2019-01-01,1082.042169,1092.660815,1079.916814,1102.000366,1106.175188,1060.723062,1085.545368,1100.671157,1062.136740,1092.875362,965.683716,969.065554,1084.980801,1142.564632,943.483740,994.810045,1033.224703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6565,2019-12-31,857.083187,857.300648,794.855865,915.517664,868.302635,795.861315,883.878613,857.782493,795.484833,917.085861,674.891174,742.331990,797.964290,830.606248,780.494827,809.243998,804.655222
6566,2019-12-31,253.030010,250.985476,238.694811,219.199028,261.876577,231.080258,204.891464,251.974263,230.437425,261.569590,232.082169,210.920805,225.576610,269.498973,229.878546,235.835586,239.325998
6567,2019-12-31,1020.716908,995.641620,937.084589,1052.911798,1009.447960,935.835037,1043.610433,989.976308,932.965624,1065.400121,730.457275,852.091174,925.788409,1133.560974,872.192736,899.608764,895.495983
6568,2019-12-31,1531.675071,1483.643573,1387.421869,1655.978947,1494.673068,1388.389257,1604.714050,1476.628593,1390.081381,1568.407214,1005.392944,1286.814471,1397.071983,2627.470943,1358.919453,1380.445879,1356.416644


### Final Assembly

In [158]:
# X = adjusted_oof_preds
# X = adjusted_oof_preds_selective
# X = adjusted_oof_preds_expanded
X = adjusted_oof_preds_selective_expanded

In [159]:
y = X['num_sold']
X = X.drop(columns=['num_sold'])

In [160]:
# X_test = adjusted_test_preds
# X_test = adjusted_test_preds_selective
# X_test = adjusted_test_preds_expanded
X_test = adjusted_test_preds_selective_expanded

In [161]:
X['date'] = pd.to_datetime(X.date)

In [162]:
X['date'] = X['date'].map(dt.datetime.toordinal)
model.fit(X,y) # this is the lasso regressor

Lasso()

In [163]:
# X_test['date'] = X['date'].map(dt.datetime.toordinal)
X_test['date'] = pd.to_datetime(X_test.date)
X_test['date'] = X_test['date'].map(dt.datetime.toordinal)

In [164]:
X

,date,xgb+prophet,xgb+ridge,xgb+linear,lgb+prophet,lgb+ridge,lgb+linear,cat+prophet,cat+ridge,cat+linear,prophet,neuralprophet,ridge,linear,earth,xgboost,lightgbm,catboost
0,735599,335.405149,324.342987,326.355622,353.816288,315.109073,320.781832,342.162795,308.871192,319.407647,346.560416,329.134521,283.272258,324.188332,360.672190,343.250478,331.496226,326.054356
1,735599,507.762170,500.921801,500.216103,502.099511,502.309507,497.514037,505.468074,494.362517,498.780657,536.203586,458.008301,439.771734,503.081569,584.851806,510.616623,514.576867,509.005936
2,735599,137.081288,138.691135,139.863963,161.207516,137.984103,133.714742,149.287358,134.114025,135.374514,143.412803,145.325577,120.885683,136.667932,138.401176,148.959080,146.429925,144.894609
3,735599,596.929485,558.648326,551.420039,577.112455,555.405898,552.703990,576.479862,555.407912,550.887544,590.117165,552.571167,488.810710,555.531957,562.479785,596.463984,584.653899,569.038022
4,735599,940.653140,890.501339,883.086316,934.119140,893.043397,882.242292,919.623890,886.321175,881.045421,939.673009,781.967285,771.701702,890.469201,944.584508,907.745110,888.842963,882.892654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26293,737059,842.534291,864.388124,838.336456,924.978028,855.545431,839.722064,905.346443,841.013156,845.194035,898.322121,669.418457,723.714830,846.597603,836.947940,830.633157,814.597001,825.377960
26294,737059,255.545977,255.901804,250.816940,189.916929,261.353502,241.161049,206.318911,244.486973,242.723551,253.512355,227.158142,205.880079,241.048951,254.287204,241.115195,234.409908,245.208674
26295,737059,992.550115,982.748408,971.987512,1006.865069,980.987703,965.731059,1031.550009,956.716929,972.733624,1039.635205,715.639648,832.192362,975.785339,1058.174059,898.057926,900.745370,915.158956
26296,737059,1505.262616,1477.136661,1464.714361,1644.013144,1473.934875,1460.599837,1614.987267,1448.806267,1466.727788,1526.908216,980.234009,1255.885410,1468.776593,2036.549357,1397.507711,1419.978895,1401.414363


In [165]:
X_test

,date,xgb+prophet,xgb+ridge,xgb+linear,lgb+prophet,lgb+ridge,lgb+linear,cat+prophet,cat+ridge,cat+linear,prophet,neuralprophet,ridge,linear,earth,xgboost,lightgbm,catboost
0,737060,388.622891,404.007541,394.495622,403.493065,403.376884,374.658654,403.252346,400.671789,374.684934,402.296576,383.543457,359.817081,392.328390,417.102143,349.759459,373.034564,379.702323
1,737060,610.880664,627.093405,620.436869,614.474655,634.724575,607.037695,612.147149,634.620835,605.461973,631.642099,551.044983,560.783324,620.092467,693.844970,549.911123,578.563016,589.688285
2,737060,160.766967,164.596231,165.280088,185.453661,175.091284,157.180923,187.572802,173.993223,157.351444,169.992094,173.065674,153.552940,166.788184,181.642807,152.988182,165.419045,168.116727
3,737060,676.305636,695.533558,675.371287,674.812040,705.677250,652.725247,673.996886,702.659724,652.858116,684.559821,648.783691,622.237644,675.674104,594.166542,622.932486,651.424049,668.232707
4,737060,1082.042169,1092.660815,1079.916814,1102.000366,1106.175188,1060.723062,1085.545368,1100.671157,1062.136740,1092.875362,965.683716,969.065554,1084.980801,1142.564632,943.483740,994.810045,1033.224703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6565,737424,857.083187,857.300648,794.855865,915.517664,868.302635,795.861315,883.878613,857.782493,795.484833,917.085861,674.891174,742.331990,797.964290,830.606248,780.494827,809.243998,804.655222
6566,737424,253.030010,250.985476,238.694811,219.199028,261.876577,231.080258,204.891464,251.974263,230.437425,261.569590,232.082169,210.920805,225.576610,269.498973,229.878546,235.835586,239.325998
6567,737424,1020.716908,995.641620,937.084589,1052.911798,1009.447960,935.835037,1043.610433,989.976308,932.965624,1065.400121,730.457275,852.091174,925.788409,1133.560974,872.192736,899.608764,895.495983
6568,737424,1531.675071,1483.643573,1387.421869,1655.978947,1494.673068,1388.389257,1604.714050,1476.628593,1390.081381,1568.407214,1005.392944,1286.814471,1397.071983,2627.470943,1358.919453,1380.445879,1356.416644


In [166]:
for col in X.columns:
    if col not in X_test.columns:
        print(col)

In [167]:
lasso_test_preds = model.predict(X_test)

In [168]:
lasso_tv_preds = model.predict(X)

Naive SMAPE was 4.152715109788985

Below had been 4.384919101043075 with XGBoost included

In [169]:
SMAPE(y_pred=lasso_tv_preds, y_true=y)

4.34354975241337

Below had been 4.27663983822924 with XGBoost included

In [170]:
ridge = Ridge()
ridge.fit(X,y)
ridge_test_preds = ridge.predict(X_test)
ridge_tv_preds = ridge.predict(X)
SMAPE(y_pred=ridge_tv_preds, y_true=y)


4.138474820600068

In [171]:
sample_df = pd.read_csv(datapath/'sample_submission.csv')

In [172]:
sample_df.loc[:, 'num_sold'] = ridge_test_preds

In [173]:
sample_df.head()

,row_id,num_sold
0,26298,394.012434
1,26299,617.972594
2,26300,166.368232
3,26301,676.177680
4,26302,1085.404238


In [174]:
sample_df.to_csv(subpath/f"20220127_specialmix_forecasts+Earth-Big3-GBMs-selective_residuals-some_passthru-ridge_preds.csv", index=False)
# sample_df.to_csv(subpath/f"{wandb_config['name']}_3level-X_orig+KMeans8+synth-GBM-stack_ensemble_preds.csv", index=False)

In [175]:
# sample_df = pd.read_csv(subpath/'LB-4.40223__20220124_forecasts_lasso_preds.csv')

In [176]:
sample_df

,row_id,num_sold
0,26298,394.012434
1,26299,617.972594
2,26300,166.368232
3,26301,676.177680
4,26302,1085.404238
...,...,...
6565,32863,798.566362
6566,32864,216.971152
6567,32865,923.824266
6568,32866,1413.388152


In [177]:
sample_df['num_sold'] = sample_df['num_sold'].apply(round)
sample_df

,row_id,num_sold
0,26298,394
1,26299,618
2,26300,166
3,26301,676
4,26302,1085
...,...,...
6565,32863,799
6566,32864,217
6567,32865,924
6568,32866,1413


In [178]:
sample_df.to_csv(subpath/'20220127_specialmix_forecasts+Earth-Big3-GBMs-selective_residuals-some_passthru-ridge_preds_rounded.csv',index=False)